
<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>
Introduction
In this project I explore, segment, and cluster the neighborhoods in the city of Toronto. The neighborhood data though is not readily available on the internet.
For the Toronto neighborhood data, a Wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M exists that has all the information we need to explore and cluster the neighborhoods in Toronto. We will have to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format.
Once the data is in a structured format, we can start the analysis to explore and cluster the neighborhoods in the city of Toronto.
We build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.
The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood. We only process the cells that have an assigned borough. We ignore cells with a borough that is Not assigned. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
There are different website scraping libraries and packages in Python. One of the most common packages is BeautifulSoup and we will use it in this project. Package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/
Before we get the data and start exploring it, let's download all the dependencies that we will need.
In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from bs4 import BeautifulSoup
import lxml
print('Libraries imported.')
Libraries imported.
Download, scrape and wrangle
In [2]:
# download data and parse it:
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(r.text, 'html.parser')
table=soup.find('table', attrs={'class':'wikitable sortable'})

#get headers:
headers=table.findAll('th')
for i, head in enumerate(headers): headers[i]=str(headers[i]).replace("<th>","").replace("</th>","").replace("\n","")

#Find all items and skip first one:
rows=table.findAll('tr')
rows=rows[1:len(rows)]

# skip all meta symbols and line feeds between rows:
for i, row in enumerate(rows): rows[i] = str(rows[i]).replace("\n</td></tr>","").replace("<tr>\n<td>","")

# make dataframe, expand rows and drop the old one:
df=pd.DataFrame(rows)
df[headers] = df[0].str.split("</td>\n<td>", n = 2, expand = True) 
df.drop(columns=[0],inplace=True)

# skip not assigned boroughs:
df = df.drop(df[(df.Borough == "Not assigned")].index)
# give "Not assigned" Neighborhoods same name as Borough:
df.Neighbourhood.replace("Not assigned", df.Borough, inplace=True)

# copy Borough value to Neighborhood if NaN:
df.Neighbourhood.fillna(df.Borough, inplace=True)
# drop duplicate rows:
df=df.drop_duplicates()

# extract titles from columns
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

df.update(
    df.Borough.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

# delete Toronto annotation from Neighbourhood:
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace(", Toronto",""))
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace("\(Toronto\)",""))

# combine multiple neighborhoods with the same post code
df2 = pd.DataFrame({'Postcode':df.Postcode.unique()})
df2['Borough']=pd.DataFrame(list(set(df['Borough'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=pd.Series(list(set(df['Neighbourhood'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=df2['Neighborhood'].apply(lambda x: ', '.join(x))
df2.dtypes

df2.head()
Out[2]:
Postcode	Borough	Neighborhood
0	M3A	North York	Parkwoods
1	M4A	North York	Victoria Village
2	M5A	Downtown Toronto	Harbourfront , Regent Park
3	M6A	North York	Lawrence Manor, Lawrence Heights
4	M7A	Queen's Park (Toronto)	Queen's Park
In [3]:
#add Geo-spatial data
dfll= pd.read_csv("http://cocl.us/Geospatial_data")
dfll.rename(columns={'Postal Code':'Postcode'}, inplace=True)
dfll.set_index("Postcode")
df2.set_index("Postcode")
toronto_data=pd.merge(df2, dfll)
toronto_data.head()
Out[3]:
Postcode	Borough	Neighborhood	Latitude	Longitude
0	M3A	North York	Parkwoods	43.753259	-79.329656
1	M4A	North York	Victoria Village	43.725882	-79.315572
2	M5A	Downtown Toronto	Harbourfront , Regent Park	43.654260	-79.360636
3	M6A	North York	Lawrence Manor, Lawrence Heights	43.718518	-79.464763
4	M7A	Queen's Park (Toronto)	Queen's Park	43.662301	-79.389494
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent to_explorer, as shown below.
In [4]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, ON, Canada are {}, {}.'.format(latitude, longitude))
The geograpical coordinate of Toronto, ON, Canada are 43.653963, -79.387207.
Create a map of Toronto with neighborhoods superimposed on top.
In [5]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto
Out[5]:
Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.
Define Foursquare Credentials and Version
In [6]:
CLIENT_ID = 'JBREGZ4UNA53HX43WMAD4TQ2X2XJWMX5DPHEZEIZHQA0ACNP' # your Foursquare ID
CLIENT_SECRET = 'VNS40KF3V4MGSWWAV0IGQINZIGIT1EQKNCWBFPOS3QF1JMOJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
Your credentails:
CLIENT_ID: JBREGZ4UNA53HX43WMAD4TQ2X2XJWMX5DPHEZEIZHQA0ACNP
CLIENT_SECRET:VNS40KF3V4MGSWWAV0IGQINZIGIT1EQKNCWBFPOS3QF1JMOJ
Let's explore the first neighborhood in our dataframe.
Get the neighborhood's name.
In [7]:
toronto_data.loc[0, 'Neighborhood']
Out[7]:
'Parkwoods'
Get the neighborhood's latitude and longitude values.
In [8]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))
Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.
Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.
First, let's create the GET request URL. Name your URL url.
In [9]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
Out[9]:
'https://api.foursquare.com/v2/venues/explore?&client_id=JBREGZ4UNA53HX43WMAD4TQ2X2XJWMX5DPHEZEIZHQA0ACNP&client_secret=VNS40KF3V4MGSWWAV0IGQINZIGIT1EQKNCWBFPOS3QF1JMOJ&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'
Send the GET request and examine the resutls
In [10]:
results = requests.get(url).json()
results
Out[10]:
{'meta': {'code': 200, 'requestId': '5c6e961fdd57977bd5b8a945'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.751976046055574,
          'lng': -79.33214044722958}],
        'distance': 245,
        'cc': 'CA',
        'city': 'Toronto',
        'state': 'ON',
        'country': 'Canada',
        'formattedAddress': ['Toronto', 'Toronto ON', 'Canada']},
       'categories': [{'id': '4bf58dd8d48988d163941735',
         'name': 'Park',
         'pluralName': 'Parks',
         'shortName': 'Park',
         'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
          'suffix': '.png'},
         'primary': True}],
       'photos': {'count': 0, 'groups': []}},
      'referralId': 'e-0-4e8d9dcdd5fbbbb6b3003c7b-0'},
     {'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e6696b6d16433b9ffff47c3',
       'name': 'KFC',
       'location': {'lat': 43.75438666345904,
        'lng': -79.3330206627504,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.75438666345904,
          'lng': -79.3330206627504}],
        'distance': 298,
        'cc': 'CA',
        'country': 'Canada',
        'formattedAddress': ['Canada']},
       'categories': [{'id': '4bf58dd8d48988d16e941735',
         'name': 'Fast Food Restaurant',
         'pluralName': 'Fast Food Restaurants',
         'shortName': 'Fast Food',
         'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
          'suffix': '.png'},
         'primary': True}],
       'photos': {'count': 0, 'groups': []}},
      'referralId': 'e-0-4e6696b6d16433b9ffff47c3-1'},
     {'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4cb11e2075ebb60cd1c4caad',
       'name': 'Variety Store',
       'location': {'address': '29 Valley Woods Road',
        'lat': 43.75197441585782,
        'lng': -79.33311418516017,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.75197441585782,
          'lng': -79.33311418516017}],
        'distance': 312,
        'cc': 'CA',
        'city': 'Toronto',
        'state': 'ON',
        'country': 'Canada',
        'formattedAddress': ['29 Valley Woods Road', 'Toronto ON', 'Canada']},
       'categories': [{'id': '4bf58dd8d48988d1f9941735',
         'name': 'Food & Drink Shop',
         'pluralName': 'Food & Drink Shops',
         'shortName': 'Food & Drink',
         'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/foodanddrink_',
          'suffix': '.png'},
         'primary': True}],
       'photos': {'count': 0, 'groups': []}},
      'referralId': 'e-0-4cb11e2075ebb60cd1c4caad-2'}]}]}}
From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.
In [11]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
Now we are ready to clean the json and structure it into a pandas dataframe.
In [12]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()
Out[12]:
name	categories	lat	lng
0	Brookbanks Park	Park	43.751976	-79.332140
1	KFC	Fast Food Restaurant	43.754387	-79.333021
2	Variety Store	Food & Drink Shop	43.751974	-79.333114
And how many venues were returned by Foursquare?
In [13]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
3 venues were returned by Foursquare.
Explore Neighborhoods in Toronto
Let's create a function to repeat the same process to all the neighborhoods.
In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
Now write the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues.
In [15]:
# type your answer here

toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )
Parkwoods
Victoria Village
Harbourfront , Regent Park
Lawrence Manor, Lawrence Heights
Queen's Park 
Islington Avenue
Malvern, Rouge
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn, Ontario
Princess Gardens, West Deane Park, Martin Grove, Islington, Cloverdale
Highland Creek , Rouge Hill, Port Union
Don Mills South, Flemingdon Park
Woodbine Heights
St. James Town
Humewood-Cedarvale
Markland Wood, Old Burnhamthorpe, Bloordale Gardens, Eringate
Morningside, Guildwood, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Woburn
Hillcrest Village
Downsview North, Wilson Heights, Bathurst Manor
Thorncliffe Park
Adelaide, Richmond, King
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Oriole, Henry Farm
Northwood Park, York University
East Toronto
Union Station , Harbourfront East, Toronto Islands
Little Portugal, Trinity–Bellwoods
Ionview, Kennedy Park, East Birchmount Park
Bayview Village
CFB Toronto, Downsview East
Riverdale, The Danforth West
Design Exchange, Toronto Dominion Centre
Exhibition Place, Parkdale Village, Brockton
Golden Mile, Clairlea, Oakridge
Silver Hills, York Mills
Downsview
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Upwood Park, North Park, Maple Leaf Park
Humber Summit
Cliffside, Scarborough Village West, Cliffcrest
Willowdale, Newtonbrook
Downsview Central
Studio District
Bedford Park, Lawrence Manor East
Silverthorn, Mount Dennis, Keelesdale, Del Ray
Emery, Humberlea
Cliffside West, Birch Cliff
Willowdale South
Downsview Northwest
Lawrence Park
Roselawn
The Junction North, Runnymede
Weston
Scarborough Town Centre, Wexford Heights, Dorset Park
York Mills West
Davisville North
Forest Hill West, Forest Hill North
High Park, The Junction South
Westmount
Wexford, Maryvale
Willowdale West
North Toronto West
Yorkville, The Annex, North Midtown
Parkdale, Roncesvalles
Canada Post Gateway Processing Centre
Richview Gardens, Kingsview Village, St. Phillips, Martin Grove Gardens
Agincourt
Davisville
University of Toronto, Harbord
Swansea, Runnymede
Sullivan, Tam O'Shanter – Sullivan, Clarks Corners
Summerhill East, Moore Park
Grange Park , Kensington Market, Chinatown
Agincourt North, Steeles East, L'Amoreaux East, Milliken, Ontario
Summerhill West, Forest Hill SE, Deer Park, South Hill, Rathnelly
South Niagara, Bathurst Quay, King and Spadina, Railway Lands, CN Tower, Harbourfront West, Island airport
Mimico South, New Toronto, Humber Bay Shores
Mount Olive-Silverstone-Jamestown, South Steeles, Thistletown, Silverstone, Beaumond Heights, Albion Gardens, Humbergate
L'Amoreaux West, Steeles West
Rosedale
Stn A PO Boxes 25 The Esplanade
Alderwood, Long Branch
Northwest
Upper Rouge
St. James Town, Cabbagetown
Underground city, First Canadian Place
The Kingsway, Montgomery Road, Old Mill North
Church and Wellesley
Business Reply Mail Processing Centre 969 Eastern
Old Mill, Sunnylea, Kingsway Park South East, Fairmont Royal York Hotel, Humber Bay, The Queensway, Kingsmills Park, Mimico
Bloor, Kingsway Park South West, Royal York South West, The Queensway, Mimico
Let's check the size of the resulting dataframe
In [16]:
print(toronto_venues.shape)
toronto_venues.head()
(2254, 7)
Out[16]:
Neighborhood	Neighborhood Latitude	Neighborhood Longitude	Venue	Venue Latitude	Venue Longitude	Venue Category
0	Parkwoods	43.753259	-79.329656	Brookbanks Park	43.751976	-79.332140	Park
1	Parkwoods	43.753259	-79.329656	KFC	43.754387	-79.333021	Fast Food Restaurant
2	Parkwoods	43.753259	-79.329656	Variety Store	43.751974	-79.333114	Food & Drink Shop
3	Victoria Village	43.725882	-79.315572	Victoria Village Arena	43.723481	-79.315635	Hockey Arena
4	Victoria Village	43.725882	-79.315572	Portugril	43.725819	-79.312785	Portuguese Restaurant
Let's check how many venues were returned for each neighborhood
In [17]:
toronto_venues.groupby('Neighborhood').count()
Out[17]:
Neighborhood Latitude	Neighborhood Longitude	Venue	Venue Latitude	Venue Longitude	Venue Category
Neighborhood						
Adelaide, Richmond, King	100	100	100	100	100	100
Agincourt	4	4	4	4	4	4
Agincourt North, Steeles East, L'Amoreaux East, Milliken, Ontario	2	2	2	2	2	2
Alderwood, Long Branch	9	9	9	9	9	9
Bayview Village	4	4	4	4	4	4
Bedford Park, Lawrence Manor East	26	26	26	26	26	26
Berczy Park	54	54	54	54	54	54
Bloor, Kingsway Park South West, Royal York South West, The Queensway, Mimico	14	14	14	14	14	14
Business Reply Mail Processing Centre 969 Eastern	18	18	18	18	18	18
CFB Toronto, Downsview East	4	4	4	4	4	4
Caledonia-Fairbanks	6	6	6	6	6	6
Canada Post Gateway Processing Centre	11	11	11	11	11	11
Central Bay Street	82	82	82	82	82	82
Christie	16	16	16	16	16	16
Church and Wellesley	87	87	87	87	87	87
Cliffside West, Birch Cliff	4	4	4	4	4	4
Cliffside, Scarborough Village West, Cliffcrest	2	2	2	2	2	2
Commerce Court, Victoria Hotel	100	100	100	100	100	100
Davisville	34	34	34	34	34	34
Davisville North	8	8	8	8	8	8
Design Exchange, Toronto Dominion Centre	100	100	100	100	100	100
Don Mills North	5	5	5	5	5	5
Don Mills South, Flemingdon Park	22	22	22	22	22	22
Dovercourt Village, Dufferin	20	20	20	20	20	20
Downsview	5	5	5	5	5	5
Downsview Central	4	4	4	4	4	4
Downsview North, Wilson Heights, Bathurst Manor	17	17	17	17	17	17
Downsview Northwest	4	4	4	4	4	4
East Toronto	4	4	4	4	4	4
Emery, Humberlea	2	2	2	2	2	2
Exhibition Place, Parkdale Village, Brockton	18	18	18	18	18	18
Fairview, Oriole, Henry Farm	65	65	65	65	65	65
Forest Hill West, Forest Hill North	5	5	5	5	5	5
Glencairn, Ontario	6	6	6	6	6	6
Golden Mile, Clairlea, Oakridge	11	11	11	11	11	11
Grange Park , Kensington Market, Chinatown	99	99	99	99	99	99
Harbourfront , Regent Park	48	48	48	48	48	48
High Park, The Junction South	24	24	24	24	24	24
Highland Creek , Rouge Hill, Port Union	2	2	2	2	2	2
Hillcrest Village	4	4	4	4	4	4
Humber Summit	2	2	2	2	2	2
Humewood-Cedarvale	4	4	4	4	4	4
Ionview, Kennedy Park, East Birchmount Park	6	6	6	6	6	6
L'Amoreaux West, Steeles West	16	16	16	16	16	16
Lawrence Manor, Lawrence Heights	15	15	15	15	15	15
Lawrence Park	5	5	5	5	5	5
Leaside	32	32	32	32	32	32
Little Portugal, Trinity–Bellwoods	63	63	63	63	63	63
Malvern, Rouge	1	1	1	1	1	1
Markland Wood, Old Burnhamthorpe, Bloordale Gardens, Eringate	6	6	6	6	6	6
Mimico South, New Toronto, Humber Bay Shores	16	16	16	16	16	16
Morningside, Guildwood, West Hill	7	7	7	7	7	7
Mount Olive-Silverstone-Jamestown, South Steeles, Thistletown, Silverstone, Beaumond Heights, Albion Gardens, Humbergate	10	10	10	10	10	10
North Toronto West	21	21	21	21	21	21
Northwest	3	3	3	3	3	3
Northwood Park, York University	5	5	5	5	5	5
Old Mill, Sunnylea, Kingsway Park South East, Fairmont Royal York Hotel, Humber Bay, The Queensway, Kingsmills Park, Mimico	1	1	1	1	1	1
Parkdale, Roncesvalles	16	16	16	16	16	16
Parkwoods	3	3	3	3	3	3
Princess Gardens, West Deane Park, Martin Grove, Islington, Cloverdale	2	2	2	2	2	2
Queen's Park	43	43	43	43	43	43
Richview Gardens, Kingsview Village, St. Phillips, Martin Grove Gardens	3	3	3	3	3	3
Riverdale, The Danforth West	41	41	41	41	41	41
Rosedale	4	4	4	4	4	4
Roselawn	1	1	1	1	1	1
Ryerson, Garden District	100	100	100	100	100	100
Scarborough Town Centre, Wexford Heights, Dorset Park	7	7	7	7	7	7
Scarborough Village	2	2	2	2	2	2
Silver Hills, York Mills	1	1	1	1	1	1
Silverthorn, Mount Dennis, Keelesdale, Del Ray	4	4	4	4	4	4
South Niagara, Bathurst Quay, King and Spadina, Railway Lands, CN Tower, Harbourfront West, Island airport	14	14	14	14	14	14
St. James Town	100	100	100	100	100	100
St. James Town, Cabbagetown	48	48	48	48	48	48
Stn A PO Boxes 25 The Esplanade	94	94	94	94	94	94
Studio District	39	39	39	39	39	39
Sullivan, Tam O'Shanter – Sullivan, Clarks Corners	9	9	9	9	9	9
Summerhill East, Moore Park	4	4	4	4	4	4
Summerhill West, Forest Hill SE, Deer Park, South Hill, Rathnelly	15	15	15	15	15	15
Swansea, Runnymede	40	40	40	40	40	40
The Beaches	4	4	4	4	4	4
The Beaches West, India Bazaar	19	19	19	19	19	19
The Junction North, Runnymede	4	4	4	4	4	4
The Kingsway, Montgomery Road, Old Mill North	3	3	3	3	3	3
Thorncliffe Park	16	16	16	16	16	16
Underground city, First Canadian Place	100	100	100	100	100	100
Union Station , Harbourfront East, Toronto Islands	100	100	100	100	100	100
University of Toronto, Harbord	35	35	35	35	35	35
Upwood Park, North Park, Maple Leaf Park	4	4	4	4	4	4
Victoria Village	4	4	4	4	4	4
Westmount	7	7	7	7	7	7
Weston	1	1	1	1	1	1
Wexford, Maryvale	4	4	4	4	4	4
Willowdale South	35	35	35	35	35	35
Willowdale West	5	5	5	5	5	5
Willowdale, Newtonbrook	2	2	2	2	2	2
Woburn	10	10	10	10	10	10
Woodbine Gardens, Parkview Hill	14	14	14	14	14	14
Woodbine Heights	9	9	9	9	9	9
York Mills West	2	2	2	2	2	2
Yorkville, The Annex, North Midtown	23	23	23	23	23	23
Let's find out how many unique categories can be curated from all the returned venues
In [18]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
There are 279 uniques categories.
Analyze Each Neighborhood
In [19]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()
Out[19]:
Yoga Studio	Accessories Store	Adult Boutique	Afghan Restaurant	Airport	Airport Food Court	Airport Gate	Airport Lounge	Airport Service	Airport Terminal	American Restaurant	Antique Shop	Aquarium	Arepa Restaurant	Art Gallery	Art Museum	Arts & Crafts Store	Asian Restaurant	Athletics & Sports	Auto Workshop	BBQ Joint	Baby Store	Bagel Shop	Bakery	Bank	Bar	Baseball Field	Baseball Stadium	Basketball Court	Basketball Stadium	Beach	Beer Bar	Beer Store	Belgian Restaurant	Bike Shop	Bistro	Boat or Ferry	Bookstore	Boutique	Brazilian Restaurant	Breakfast Spot	Brewery	Bridal Shop	Bubble Tea Shop	Building	Burger Joint	Burrito Place	Bus Line	Bus Station	Bus Stop	Business Service	Butcher	Cafeteria	Café	Cajun / Creole Restaurant	Candy Store	Caribbean Restaurant	Cheese Shop	Chinese Restaurant	Chocolate Shop	Church	Climbing Gym	Clothing Store	Cocktail Bar	Coffee Shop	College Arts Building	College Auditorium	College Gym	College Rec Center	College Stadium	Comfort Food Restaurant	Comic Shop	Concert Hall	Construction & Landscaping	Convenience Store	Cosmetics Shop	Coworking Space	Creperie	Cuban Restaurant	Cupcake Shop	Curling Ice	Dance Studio	Deli / Bodega	Department Store	Dessert Shop	Dim Sum Restaurant	Diner	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restaurant	Electronics Store	Empanada Restaurant	Ethiopian Restaurant	Event Space	Falafel Restaurant	Farmers Market	Fast Food Restaurant	Field	Filipino Restaurant	Fish & Chips Shop	Fish Market	Flea Market	Flower Shop	Food	Food & Drink Shop	Food Court	Food Truck	Fountain	Fraternity House	French Restaurant	Fried Chicken Joint	Frozen Yogurt Shop	Fruit & Vegetable Store	Furniture / Home Store	Gaming Cafe	Garden	Garden Center	Gastropub	Gay Bar	General Entertainment	General Travel	German Restaurant	Gift Shop	Gluten-free Restaurant	Golf Course	Gourmet Shop	Greek Restaurant	Grocery Store	Gym	Gym / Fitness Center	Gym Pool	Hakka Restaurant	Harbor / Marina	Hardware Store	Health & Beauty Service	Health Food Store	Historic Site	History Museum	Hobby Shop	Hockey Arena	Hookah Bar	Hospital	Hostel	Hotel	Hotel Bar	Hotpot Restaurant	Ice Cream Shop	Indian Restaurant	Indie Movie Theater	Indonesian Restaurant	Indoor Play Area	Intersection	Irish Pub	Italian Restaurant	Japanese Restaurant	Jazz Club	Jewelry Store	Jewish Restaurant	Juice Bar	Korean Restaurant	Lake	Latin American Restaurant	Light Rail Station	Lingerie Store	Liquor Store	Lounge	Luggage Store	Mac & Cheese Joint	Malay Restaurant	Market	Martial Arts Dojo	Massage Studio	Medical Center	Mediterranean Restaurant	Men's Store	Metro Station	Mexican Restaurant	Middle Eastern Restaurant	Miscellaneous Shop	Mobile Phone Shop	Modern European Restaurant	Molecular Gastronomy Restaurant	Monument / Landmark	Motel	Movie Theater	Museum	Music Store	Music Venue	Nail Salon	Neighborhood	New American Restaurant	Nightclub	Noodle House	Office	Opera House	Optical Shop	Organic Grocery	Other Great Outdoors	Outdoor Sculpture	Park	Performing Arts Venue	Persian Restaurant	Pet Store	Pharmacy	Piano Bar	Pizza Place	Plane	Playground	Plaza	Poke Place	Pool	Portuguese Restaurant	Poutine Place	Print Shop	Pub	Ramen Restaurant	Record Shop	Recording Studio	Rental Car Location	Restaurant	River	Rock Climbing Spot	Sake Bar	Salad Place	Salon / Barbershop	Sandwich Place	Scenic Lookout	Sculpture Garden	Seafood Restaurant	Shoe Store	Shopping Mall	Shopping Plaza	Skate Park	Skating Rink	Smoke Shop	Smoothie Shop	Snack Place	Soccer Field	Social Club	Soup Place	Southern / Soul Food Restaurant	Spa	Speakeasy	Sporting Goods Shop	Sports Bar	Stadium	Stationery Store	Steakhouse	Strip Club	Supermarket	Supplement Shop	Sushi Restaurant	Swim School	Taco Place	Tailor Shop	Taiwanese Restaurant	Tanning Salon	Tapas Restaurant	Tea Room	Tennis Court	Thai Restaurant	Theater	Theme Restaurant	Thrift / Vintage Store	Toy / Game Store	Trail	Train Station	Vegetarian / Vegan Restaurant	Video Game Store	Video Store	Vietnamese Restaurant	Warehouse Store	Wine Bar	Wings Joint	Women's Store
0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	Parkwoods	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	Parkwoods	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
2	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	Parkwoods	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
3	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	Victoria Village	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
4	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	Victoria Village	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
And let's examine the new dataframe size.
In [20]:
toronto_onehot.shape
Out[20]:
(2254, 279)
Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
In [21]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped
Out[21]:
Neighborhood	Yoga Studio	Accessories Store	Adult Boutique	Afghan Restaurant	Airport	Airport Food Court	Airport Gate	Airport Lounge	Airport Service	Airport Terminal	American Restaurant	Antique Shop	Aquarium	Arepa Restaurant	Art Gallery	Art Museum	Arts & Crafts Store	Asian Restaurant	Athletics & Sports	Auto Workshop	BBQ Joint	Baby Store	Bagel Shop	Bakery	Bank	Bar	Baseball Field	Baseball Stadium	Basketball Court	Basketball Stadium	Beach	Beer Bar	Beer Store	Belgian Restaurant	Bike Shop	Bistro	Boat or Ferry	Bookstore	Boutique	Brazilian Restaurant	Breakfast Spot	Brewery	Bridal Shop	Bubble Tea Shop	Building	Burger Joint	Burrito Place	Bus Line	Bus Station	Bus Stop	Business Service	Butcher	Cafeteria	Café	Cajun / Creole Restaurant	Candy Store	Caribbean Restaurant	Cheese Shop	Chinese Restaurant	Chocolate Shop	Church	Climbing Gym	Clothing Store	Cocktail Bar	Coffee Shop	College Arts Building	College Auditorium	College Gym	College Rec Center	College Stadium	Comfort Food Restaurant	Comic Shop	Concert Hall	Construction & Landscaping	Convenience Store	Cosmetics Shop	Coworking Space	Creperie	Cuban Restaurant	Cupcake Shop	Curling Ice	Dance Studio	Deli / Bodega	Department Store	Dessert Shop	Dim Sum Restaurant	Diner	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restaurant	Electronics Store	Empanada Restaurant	Ethiopian Restaurant	Event Space	Falafel Restaurant	Farmers Market	Fast Food Restaurant	Field	Filipino Restaurant	Fish & Chips Shop	Fish Market	Flea Market	Flower Shop	Food	Food & Drink Shop	Food Court	Food Truck	Fountain	Fraternity House	French Restaurant	Fried Chicken Joint	Frozen Yogurt Shop	Fruit & Vegetable Store	Furniture / Home Store	Gaming Cafe	Garden	Garden Center	Gastropub	Gay Bar	General Entertainment	General Travel	German Restaurant	Gift Shop	Gluten-free Restaurant	Golf Course	Gourmet Shop	Greek Restaurant	Grocery Store	Gym	Gym / Fitness Center	Gym Pool	Hakka Restaurant	Harbor / Marina	Hardware Store	Health & Beauty Service	Health Food Store	Historic Site	History Museum	Hobby Shop	Hockey Arena	Hookah Bar	Hospital	Hostel	Hotel	Hotel Bar	Hotpot Restaurant	Ice Cream Shop	Indian Restaurant	Indie Movie Theater	Indonesian Restaurant	Indoor Play Area	Intersection	Irish Pub	Italian Restaurant	Japanese Restaurant	Jazz Club	Jewelry Store	Jewish Restaurant	Juice Bar	Korean Restaurant	Lake	Latin American Restaurant	Light Rail Station	Lingerie Store	Liquor Store	Lounge	Luggage Store	Mac & Cheese Joint	Malay Restaurant	Market	Martial Arts Dojo	Massage Studio	Medical Center	Mediterranean Restaurant	Men's Store	Metro Station	Mexican Restaurant	Middle Eastern Restaurant	Miscellaneous Shop	Mobile Phone Shop	Modern European Restaurant	Molecular Gastronomy Restaurant	Monument / Landmark	Motel	Movie Theater	Museum	Music Store	Music Venue	Nail Salon	New American Restaurant	Nightclub	Noodle House	Office	Opera House	Optical Shop	Organic Grocery	Other Great Outdoors	Outdoor Sculpture	Park	Performing Arts Venue	Persian Restaurant	Pet Store	Pharmacy	Piano Bar	Pizza Place	Plane	Playground	Plaza	Poke Place	Pool	Portuguese Restaurant	Poutine Place	Print Shop	Pub	Ramen Restaurant	Record Shop	Recording Studio	Rental Car Location	Restaurant	River	Rock Climbing Spot	Sake Bar	Salad Place	Salon / Barbershop	Sandwich Place	Scenic Lookout	Sculpture Garden	Seafood Restaurant	Shoe Store	Shopping Mall	Shopping Plaza	Skate Park	Skating Rink	Smoke Shop	Smoothie Shop	Snack Place	Soccer Field	Social Club	Soup Place	Southern / Soul Food Restaurant	Spa	Speakeasy	Sporting Goods Shop	Sports Bar	Stadium	Stationery Store	Steakhouse	Strip Club	Supermarket	Supplement Shop	Sushi Restaurant	Swim School	Taco Place	Tailor Shop	Taiwanese Restaurant	Tanning Salon	Tapas Restaurant	Tea Room	Tennis Court	Thai Restaurant	Theater	Theme Restaurant	Thrift / Vintage Store	Toy / Game Store	Trail	Train Station	Vegetarian / Vegan Restaurant	Video Game Store	Video Store	Vietnamese Restaurant	Warehouse Store	Wine Bar	Wings Joint	Women's Store
0	Adelaide, Richmond, King	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.040000	0.000000	0.00	0.000000	0.010000	0.010000	0.000000	0.030000	0.000000	0.000000	0.000000	0.0000	0.000000	0.030000	0.000000	0.030000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020000	0.000000	0.01	0.020000	0.000000	0.000000	0.000000	0.01	0.020000	0.010000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.050000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.030000	0.010000	0.060000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.020000	0.00	0.000000	0.020000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.010000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.020000	0.000000	0.000000	0.01	0.00	0.000000	0.01	0.00	0.000000	0.010000	0.000000	0.030000	0.010000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.030000	0.000000	0.000000	0.010000	0.010000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020000	0.010000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.01	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.010000	0.000000	0.010000	0.010000	0.01	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.020000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.010000	0.0	0.000000	0.000000	0.010000	0.000000	0.000000	0.030000	0.000000	0.000000	0.000000	0.01000	0.010000	0.000000	0.00	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.040000	0.000000	0.000000	0.000000	0.020000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.040000	0.010000	0.000000	0.000000	0.000000	0.00000	0.000000	0.010000	0.000000	0.000000	0.000000	0.0000	0.010000	0.000000	0.010000
1	Agincourt	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.250000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
2	Agincourt North, Steeles East, L'Amoreaux East...	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.500000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.500000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
3	Alderwood, Long Branch	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.111111	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.111111	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.111111	0.0	0.222222	0.000000	0.000000	0.000000	0.000000	0.111111	0.000000	0.000000	0.0	0.111111	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.111111	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.111111	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
4	Bayview Village	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.250000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.250000	0.000000	0.000000	0.000000	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
5	Bedford Park, Lawrence Manor East	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.038462	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.038462	0.0	0.038462	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.076923	0.000000	0.000000	0.000000	0.00	0.00	0.038462	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.038462	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.076923	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.038462	0.038462	0.000000	0.000000	0.0000	0.0	0.000000	0.038462	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.038462	0.000	0.000000	0.000000	0.000000	0.000000	0.115385	0.000000	0.000000	0.000000	0.000000	0.038462	0.000000	0.00	0.000000	0.000000	0.00	0.038462	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.038462	0.0	0.038462	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.038462	0.000000	0.000000	0.000000	0.000000	0.038462	0.000000	0.000000	0.000000	0.00000	0.000000	0.038462	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.038462	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.076923	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
6	Berczy Park	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.018519	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.018519	0.037037	0.000000	0.000000	0.000000	0.00	0.00	0.018519	0.018519	0.037037	0.000000	0.018519	0.000000	0.018519	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.037037	0.000000	0.000000	0.000000	0.037037	0.000000	0.000000	0.000000	0.000000	0.018519	0.055556	0.074074	0.000000	0.000000	0.000000	0.00	0.00	0.018519	0.000000	0.018519	0.00	0.000000	0.018519	0.000000	0.018519	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.018519	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.037037	0.000000	0.00	0.000000	0.000000	0.018519	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.018519	0.000000	0.018519	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.018519	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.018519	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.018519	0.037037	0.000000	0.018519	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.018519	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.018519	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.018519	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.037037	0.000000	0.000000	0.000000	0.000000	0.055556	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.037037	0.000000	0.018519	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.037037	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.018519	0.000000	0.00	0.000000	0.018519	0.00	0.018519	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
7	Bloor, Kingsway Park South West, Royal York So...	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.071429	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.071429	0.071429	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.071429	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.071429	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.071429	0.00	0.000000	0.000000	0.000000	0.000000	0.071429	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.071429	0.071429	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.071429	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.071429	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.071429	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.071429	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.071429	0.000000
8	Business Reply Mail Processing Centre 969 Eastern	0.055556	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.055556	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.055556	0.000000	0.000000	0.00	0.000000	0.055556	0.000000	0.000000	0.00	0.00	0.055556	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.055556	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.055556	0.055556	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.055556	0.055556	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.111111	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.055556	0.000000	0.000000	0.000000	0.000000	0.0	0.055556	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.055556	0.000000	0.055556	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.055556	0.000000	0.055556	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
9	CFB Toronto, Downsview East	0.000000	0.000000	0.000000	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.25	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.250000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
10	Caledonia-Fairbanks	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.166667	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.166667	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.333333	0.000000	0.000000	0.000000	0.166667	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.166667
11	Canada Post Gateway Processing Centre	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.090909	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.090909	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.181818	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.090909	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.090909	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.181818	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.090909	0.000000	0.000000	0.000000	0.090909	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.090909	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
12	Central Bay Street	0.012195	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.012195	0.000000	0.00	0.000000	0.000000	0.012195	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.012195	0.000000	0.036585	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.024390	0.00	0.036585	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.060976	0.000000	0.000000	0.000000	0.000000	0.036585	0.000000	0.000000	0.000000	0.012195	0.000000	0.158537	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.012195	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.012195	0.012195	0.000000	0.012195	0.012195	0.000000	0.000000	0.012195	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.012195	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.012195	0.012195	0.000000	0.00000	0.012195	0.000000	0.000000	0.000000	0.012195	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.012195	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.024390	0.024390	0.000	0.000000	0.000000	0.000000	0.000000	0.048780	0.012195	0.000000	0.000000	0.000000	0.000000	0.012195	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.012195	0.012195	0.000000	0.012195	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.012195	0.00	0.000000	0.000000	0.00	0.000000	0.012195	0.000000	0.000000	0.000000	0.000000	0.0	0.012195	0.000000	0.000000	0.000000	0.012195	0.000000	0.012195	0.000000	0.0	0.000000	0.012195	0.000000	0.000000	0.000000	0.012195	0.000000	0.000000	0.000000	0.02439	0.000000	0.024390	0.00	0.000000	0.012195	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.012195	0.000000	0.000000	0.000000	0.000000	0.000000	0.024390	0.000000	0.000000	0.000000	0.000000	0.000000	0.012195	0.000000	0.000000	0.000000	0.012195	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.012195	0.00	0.024390	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.012195	0.000000	0.012195	0.000000	0.0000	0.012195	0.000000	0.000000
13	Christie	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.062500	0.000000	0.000000	0.0625	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.187500	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.062500	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.062500	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.062500	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.187500	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.062500	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.062500	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.125000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.062500	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
14	Church and Wellesley	0.011494	0.000000	0.011494	0.011494	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.011494	0.000000	0.00	0.000000	0.000000	0.000000	0.011494	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.011494	0.000000	0.00	0.011494	0.000000	0.000000	0.022989	0.00	0.034483	0.011494	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.022989	0.000000	0.000000	0.011494	0.000000	0.011494	0.000000	0.000000	0.000000	0.000000	0.000000	0.057471	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.011494	0.000000	0.000000	0.000000	0.011494	0.000000	0.000000	0.000000	0.000000	0.011494	0.000000	0.011494	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.011494	0.000000	0.000000	0.000000	0.022989	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.011494	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.022989	0.045977	0.011494	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.011494	0.011494	0.000000	0.0000	0.0	0.000000	0.000000	0.011494	0.000000	0.000000	0.000000	0.011494	0.00	0.00	0.000000	0.000000	0.011494	0.000000	0.000000	0.011494	0.011494	0.000	0.000000	0.000000	0.011494	0.000000	0.000000	0.068966	0.000000	0.000000	0.000000	0.011494	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.022989	0.022989	0.000000	0.011494	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.011494	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.011494	0.000000	0.011494	0.000000	0.000000	0.0	0.022989	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.022989	0.011494	0.000000	0.000000	0.000000	0.034483	0.000000	0.000000	0.011494	0.00000	0.011494	0.000000	0.00	0.011494	0.011494	0.000000	0.000000	0.000000	0.000000	0.000000	0.011494	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.011494	0.000000	0.000000	0.011494	0.011494	0.000000	0.000000	0.057471	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.011494	0.00	0.000000	0.011494	0.011494	0.000000	0.000000	0.00000	0.000000	0.000000	0.011494	0.000000	0.011494	0.0000	0.000000	0.011494	0.000000
15	Cliffside West, Birch Cliff	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.250000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.25	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.250000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
16	Cliffside, Scarborough Village West, Cliffcrest	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.500000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.5	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
17	Commerce Court, Victoria Hotel	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.040000	0.000000	0.00	0.000000	0.010000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.0000	0.000000	0.030000	0.000000	0.020000	0.000000	0.00	0.00	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.00	0.010000	0.000000	0.000000	0.000000	0.01	0.010000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.070000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020000	0.100000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.010000	0.00	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.030000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.010000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.010000	0.010000	0.010000	0.000000	0.010000	0.010000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.030000	0.000000	0.000000	0.01	0.00	0.000000	0.01	0.00	0.000000	0.010000	0.000000	0.030000	0.010000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.060000	0.010000	0.000000	0.010000	0.000000	0.000	0.000000	0.000000	0.000000	0.010000	0.030000	0.020000	0.010000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.01	0.0	0.000000	0.010000	0.000000	0.000000	0.0000	0.010000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.0	0.010000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.0	0.010000	0.000000	0.000000	0.000000	0.000000	0.060000	0.000000	0.000000	0.000000	0.01000	0.000000	0.010000	0.00	0.000000	0.030000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.030000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.010000	0.000000	0.00	0.000000	0.010000	0.00	0.020000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.010000	0.000000	0.000000
18	Davisville	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.029412	0.000000	0.000000	0.00	0.029412	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.058824	0.000000	0.000000	0.000000	0.000000	0.029412	0.000000	0.000000	0.000000	0.000000	0.000000	0.058824	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.029412	0.000000	0.088235	0.000000	0.029412	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.029412	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.029412	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.029412	0.029412	0.000000	0.029412	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.029412	0.000	0.000000	0.029412	0.000000	0.000000	0.058824	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.029412	0.000000	0.000000	0.000000	0.029412	0.0	0.058824	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.029412	0.000000	0.000000	0.000000	0.00000	0.000000	0.088235	0.00	0.000000	0.029412	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.058824	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.029412	0.000000	0.000000	0.000000	0.029412	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
19	Davisville North	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.125000	0.000000	0.000000	0.000000	0.00	0.125000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.125000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.125000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.125000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.125000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.125000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.125000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
20	Design Exchange, Toronto Dominion Centre	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.040000	0.000000	0.00	0.000000	0.010000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.0000	0.000000	0.010000	0.000000	0.020000	0.000000	0.00	0.00	0.010000	0.000000	0.020000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.00	0.010000	0.000000	0.000000	0.000000	0.00	0.020000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.090000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.150000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.010000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.030000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.010000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.010000	0.000000	0.000000	0.000000	0.010000	0.010000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.030000	0.000000	0.000000	0.01	0.00	0.000000	0.01	0.00	0.000000	0.010000	0.000000	0.020000	0.010000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.080000	0.010000	0.000000	0.010000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.030000	0.020000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.010000	0.000000	0.000000	0.0000	0.010000	0.000000	0.010000	0.010000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.010000	0.000000	0.000000	0.000000	0.000000	0.040000	0.000000	0.000000	0.000000	0.01000	0.000000	0.010000	0.00	0.000000	0.020000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.010000	0.010000	0.010000	0.010000	0.000000	0.000000	0.020000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.010000	0.00	0.010000	0.010000	0.000000	0.000000	0.000000	0.00000	0.010000	0.000000	0.000000	0.000000	0.000000	0.0000	0.010000	0.000000	0.000000
21	Don Mills North	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.200000	0.000000	0.000000	0.200000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.200000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.200000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.200000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
22	Don Mills South, Flemingdon Park	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.090909	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.090909	0.000000	0.045455	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.045455	0.000000	0.000000	0.000000	0.000000	0.045455	0.000000	0.000000	0.000000	0.045455	0.000000	0.090909	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.045455	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.045455	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.045455	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.045455	0.090909	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.045455	0.045455	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.045455	0.000000	0.000000	0.000000	0.00000	0.000000	0.045455	0.00	0.000000	0.000000	0.000000	0.045455	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.045455	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
23	Dovercourt Village, Dufferin	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.100000	0.050000	0.050000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.050000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.050000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.100000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.050000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.050000	0.050000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.050000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.050000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.050000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.050000	0.000000	0.000000	0.000000	0.100000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.050000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.100000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
24	Downsview	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.200000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.400000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.200000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.200000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
25	Downsview Central	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.250000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.25	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.250000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
26	Downsview North, Wilson Heights, Bathurst Manor	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.058824	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.058824	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.117647	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.058824	0.000000	0.000000	0.000000	0.058824	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.058824	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.058824	0.058824	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.058824	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.058824	0.0	0.058824	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.058824	0.000000	0.000000	0.000000	0.00000	0.000000	0.058824	0.00	0.000000	0.000000	0.000000	0.058824	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.058824	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.058824	0.000000	0.0000	0.000000	0.000000	0.000000
27	Downsview Northwest	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.250000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.000000	0.250000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.250000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
28	East Toronto	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.250000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.500000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
29	Emery, Humberlea	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.500000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.50	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
30	Exhibition Place, Parkdale Village, Brockton	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.055556	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.111111	0.000000	0.000000	0.000000	0.00	0.000000	0.055556	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.111111	0.000000	0.000000	0.055556	0.000000	0.000000	0.000000	0.000000	0.055556	0.000000	0.000000	0.111111	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.055556	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.055556	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.055556	0.055556	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.055556	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.055556	0.000000	0.055556	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.055556	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
31	Fairview, Oriole, Henry Farm	0.000000	0.015385	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.015385	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.030769	0.000000	0.000000	0.000000	0.0000	0.000000	0.030769	0.015385	0.000000	0.015385	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.015385	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.015385	0.015385	0.000000	0.030769	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.015385	0.000000	0.000000	0.015385	0.000000	0.000000	0.000000	0.123077	0.000000	0.061538	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.015385	0.015385	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.015385	0.015385	0.015385	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.030769	0.0	0.000000	0.000000	0.000000	0.000000	0.076923	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.030769	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.015385	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.015385	0.000000	0.000000	0.000000	0.015385	0.000000	0.00	0.000000	0.000000	0.00	0.015385	0.000000	0.015385	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.015385	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.015385	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.015385	0.000000	0.000000	0.030769	0.000000	0.000000	0.000000	0.00000	0.015385	0.000000	0.00	0.000000	0.000000	0.015385	0.015385	0.000000	0.000000	0.000000	0.000000	0.015385	0.000000	0.000000	0.000000	0.000000	0.000000	0.015385	0.000000	0.015385	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.015385	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.030769	0.00	0.000000	0.015385	0.000000	0.000000	0.015385	0.00000	0.000000	0.000000	0.015385	0.000000	0.000000	0.0000	0.000000	0.015385	0.015385
32	Forest Hill West, Forest Hill North	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.200000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.200000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.200000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.200000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.20000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
33	Glencairn, Ontario	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.166667	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.166667	0.000000	0.000000	0.000000	0.000000	0.0	0.333333	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.166667	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.166667	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
34	Golden Mile, Clairlea, Oakridge	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.181818	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.181818	0.090909	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.090909	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.090909	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.090909	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.090909	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.090909	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.090909	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
35	Grange Park , Kensington Market, Chinatown	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.010101	0.000000	0.000000	0.010101	0.000000	0.000000	0.000000	0.000000	0.0000	0.010101	0.040404	0.000000	0.060606	0.000000	0.00	0.00	0.000000	0.000000	0.010101	0.000000	0.010101	0.000000	0.010101	0.000000	0.000000	0.000000	0.00	0.010101	0.010101	0.000000	0.010101	0.00	0.020202	0.010101	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.070707	0.000000	0.000000	0.020202	0.010101	0.030303	0.000000	0.000000	0.000000	0.000000	0.010101	0.040404	0.000000	0.000000	0.000000	0.00	0.00	0.010101	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020202	0.020202	0.000000	0.000000	0.000000	0.010101	0.010101	0.000000	0.040404	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.010101	0.000000	0.00	0.010101	0.010101	0.010101	0.000000	0.000000	0.000	0.000000	0.010101	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.010101	0.020202	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.010101	0.000000	0.020202	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.010101	0.000000	0.000000	0.010101	0.010101	0.010101	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.010101	0.010101	0.010101	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.010101	0.000000	0.000000	0.000000	0.000000	0.000000	0.010101	0.0	0.000000	0.000000	0.000000	0.000000	0.030303	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.010101	0.000000	0.00	0.000000	0.010101	0.00	0.000000	0.010101	0.000000	0.000000	0.000000	0.010101	0.0	0.010101	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010101	0.0	0.000000	0.010101	0.010101	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010101	0.010101	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.010101	0.000000	0.000000	0.00	0.000000	0.010101	0.00	0.010101	0.000000	0.000000	0.010101	0.000000	0.00000	0.000000	0.050505	0.000000	0.000000	0.050505	0.0000	0.010101	0.000000	0.000000
36	Harbourfront , Regent Park	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020833	0.00	0.000000	0.020833	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.062500	0.020833	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.020833	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.041667	0.020833	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.062500	0.000000	0.000000	0.000000	0.000000	0.000000	0.020833	0.000000	0.000000	0.000000	0.000000	0.145833	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020833	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.020833	0.0	0.000000	0.020833	0.000000	0.020833	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020833	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.020833	0.000000	0.000000	0.020833	0.0000	0.0	0.000000	0.000000	0.000000	0.020833	0.020833	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.020833	0.000000	0.000000	0.020833	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.020833	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.041667	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.062500	0.020833	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.062500	0.000000	0.000000	0.000000	0.000000	0.041667	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.020833	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020833	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.041667	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
37	High Park, The Junction South	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.041667	0.00	0.000000	0.000000	0.000000	0.041667	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.041667	0.000000	0.041667	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.041667	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.083333	0.041667	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.041667	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.041667	0.00	0.000000	0.000000	0.000000	0.041667	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.041667	0.000000	0.00000	0.041667	0.000000	0.000000	0.000000	0.041667	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.041667	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.041667	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.083333	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.041667	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.083333	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.041667	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.041667	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.041667	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
38	Highland Creek , Rouge Hill, Port Union	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.500000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.500000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
39	Hillcrest Village	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.25	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.250000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
40	Humber Summit	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.5	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.500000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
41	Humewood-Cedarvale	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.25	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.25	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.25000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
42	Ionview, Kennedy Park, East Birchmount Park	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.166667	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.166667	0.000000	0.000000	0.000000	0.333333	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.166667	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.166667	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
43	L'Amoreaux West, Steeles West	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.062500	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.125000	0.000000	0.000000	0.000000	0.000000	0.000000	0.125000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.062500	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.125000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.062500	0.000000	0.000000	0.0625	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.062500	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0625	0.000000	0.000000	0.062500	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.062500	0.0	0.062500	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.062500	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
44	Lawrence Manor, Lawrence Heights	0.000000	0.066667	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.066667	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.066667	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.333333	0.000000	0.066667	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.066667	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.066667	0.000000	0.000000	0.000000	0.00000	0.066667	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.066667	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.066667	0.0000	0.000000	0.000000	0.066667
45	Lawrence Park	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.200000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.200000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.20	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.200000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.2	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
46	Leaside	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.031250	0.000000	0.031250	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.031250	0.000000	0.031250	0.000000	0.000000	0.000000	0.000000	0.00	0.031250	0.031250	0.000000	0.000000	0.00	0.062500	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.031250	0.000000	0.093750	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.031250	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.031250	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.031250	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.031250	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.062500	0.031250	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.031250	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.031250	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.031250	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.031250	0.000000	0.000000	0.000000	0.00000	0.000000	0.031250	0.00	0.000000	0.000000	0.000000	0.031250	0.000000	0.000000	0.000000	0.000000	0.031250	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.093750	0.031250	0.000000	0.000000	0.000000	0.000000	0.031250	0.000000	0.031250	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
47	Little Portugal, Trinity–Bellwoods	0.015873	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.015873	0.000000	0.00	0.000000	0.015873	0.000000	0.000000	0.047619	0.000000	0.000000	0.000000	0.0000	0.000000	0.031746	0.000000	0.126984	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.015873	0.000000	0.000000	0.015873	0.00	0.000000	0.015873	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.031746	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.031746	0.047619	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.015873	0.015873	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.015873	0.000000	0.015873	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.015873	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.015873	0.00	0.00	0.000000	0.015873	0.000000	0.000000	0.015873	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.015873	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.015873	0.000000	0.000000	0.000000	0.000000	0.015873	0.015873	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.015873	0.015873	0.000000	0.000000	0.0	0.000000	0.000000	0.063492	0.000000	0.000000	0.000000	0.015873	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.015873	0.0000	0.015873	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.015873	0.000000	0.000000	0.000000	0.000000	0.0	0.031746	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.015873	0.000000	0.015873	0.000000	0.000000	0.031746	0.000000	0.000000	0.000000	0.00000	0.015873	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.015873	0.000000	0.000000	0.000000	0.015873	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.015873	0.000000	0.00	0.000000	0.015873	0.000000	0.000000	0.000000	0.00000	0.000000	0.015873	0.000000	0.000000	0.031746	0.0000	0.015873	0.000000	0.000000
48	Malvern, Rouge	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	1.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
49	Markland Wood, Old Burnhamthorpe, Bloordale Ga...	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.166667	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.166667	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.166667	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.166667	0.0	0.166667	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.166667	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
50	Mimico South, New Toronto, Humber Bay Shores	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.062500	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.062500	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.125000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.062500	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.062500	0.00	0.000000	0.000000	0.000000	0.000000	0.062500	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.062500	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.062500	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.062500	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.062500	0.062500	0.0	0.062500	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.062500	0.000000	0.000000	0.000000	0.00000	0.000000	0.062500	0.00	0.000000	0.062500	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
51	Morningside, Guildwood, West Hill	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.142857	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.142857	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.142857	0.000000	0.000000	0.000000	0.142857	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.142857	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.142857	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.142857	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
52	Mount Olive-Silverstone-Jamestown, South Steel...	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.100000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.100000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.100000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.100000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.200000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.100000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.100000	0.0	0.100000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.100000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
53	North Toronto West	0.047619	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.047619	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.047619	0.000000	0.000000	0.000000	0.142857	0.000000	0.095238	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.047619	0.000000	0.047619	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.047619	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.047619	0.000000	0.047619	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.047619	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.047619	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.047619	0.000000	0.000000	0.000000	0.000000	0.00000	0.047619	0.047619	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.047619	0.000000	0.095238	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
54	Northwest	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.333333	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.333333	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.333333	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
55	Northwood Park, York University	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.200000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.200000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.2	0.000000	0.000000	0.000000	0.200000	0.000000	0.000000	0.200000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
56	Old Mill, Sunnylea, Kingsway Park South East, ...	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	1.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
57	Parkdale, Roncesvalles	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.062500	0.062500	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.062500	0.000000	0.00	0.125000	0.000000	0.000000	0.000000	0.00	0.062500	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.062500	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.062500	0.000000	0.000000	0.000000	0.000000	0.000000	0.062500	0.000000	0.000000	0.000000	0.062500	0.000000	0.000000	0.000000	0.000000	0.0625	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.125000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.062500	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.062500	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.062500	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
58	Parkwoods	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.333333	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.333333	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.333333	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
59	Princess Gardens, West Deane Park, Martin Grov...	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.500000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.5	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
60	Queen's Park	0.023256	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.023256	0.000000	0.023256	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.023256	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.023256	0.00	0.023256	0.023256	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.023256	0.000000	0.000000	0.000000	0.000000	0.023256	0.000000	0.000000	0.000000	0.000000	0.000000	0.232558	0.000000	0.023256	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.023256	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.046512	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.023256	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.046512	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.023256	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.023256	0.046512	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.023256	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.023256	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.023256	0.000000	0.023256	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.023256	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.023256	0.00	0.000000	0.023256	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.023256	0.000000	0.000000	0.000000	0.000000	0.000000	0.023256	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.046512	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.023256	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.023256	0.000000
61	Richview Gardens, Kingsview Village, St. Phill...	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.333333	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.333333	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.333333	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
62	Riverdale, The Danforth West	0.024390	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.024390	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.024390	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.048780	0.000000	0.00	0.000000	0.024390	0.000000	0.024390	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.024390	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.097561	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.024390	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.024390	0.000000	0.024390	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.02439	0.024390	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.243902	0.024390	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.073171	0.024390	0.000	0.000000	0.000000	0.000000	0.000000	0.048780	0.000000	0.000000	0.000000	0.000000	0.024390	0.000000	0.00	0.000000	0.000000	0.00	0.024390	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.024390	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.024390	0.000000	0.000000	0.000000	0.000000	0.024390	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.024390	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.02439	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
63	Rosedale	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.500000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.25000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
64	Roselawn	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	1.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
65	Ryerson, Garden District	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.00	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.0000	0.000000	0.010000	0.010000	0.020000	0.000000	0.00	0.00	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.00	0.000000	0.000000	0.000000	0.010000	0.00	0.010000	0.010000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.040000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.090000	0.000000	0.090000	0.000000	0.000000	0.000000	0.01	0.00	0.000000	0.010000	0.010000	0.00	0.000000	0.030000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.010000	0.000000	0.020000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.010000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.010000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.010000	0.010000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.01	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.020000	0.020000	0.000000	0.000000	0.000000	0.010000	0.000000	0.01	0.000000	0.000000	0.01	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.010000	0.030000	0.010000	0.000000	0.010000	0.000000	0.00	0.0	0.010000	0.000000	0.000000	0.010000	0.0000	0.000000	0.000000	0.000000	0.010000	0.00	0.000000	0.000000	0.01	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.020000	0.000000	0.000000	0.020000	0.000000	0.000000	0.000000	0.000000	0.0	0.010000	0.020000	0.000000	0.000000	0.000000	0.020000	0.000000	0.000000	0.000000	0.00000	0.000000	0.010000	0.00	0.000000	0.010000	0.010000	0.010000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.0	0.010000	0.000000	0.000000	0.01	0.000000	0.020000	0.00	0.020000	0.020000	0.000000	0.000000	0.010000	0.00000	0.000000	0.010000	0.010000	0.000000	0.010000	0.0000	0.010000	0.000000	0.000000
66	Scarborough Town Centre, Wexford Heights, Dors...	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.142857	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.142857	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.285714	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.142857	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.142857	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.142857	0.0000	0.000000	0.000000	0.000000
67	Scarborough Village	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.500000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.500000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
68	Silver Hills, York Mills	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	1.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
69	Silverthorn, Mount Dennis, Keelesdale, Del Ray	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.250000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.250000	0.000000	0.000000	0.000000	0.00000	0.000000	0.250000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
70	South Niagara, Bathurst Quay, King and Spadina...	0.000000	0.000000	0.000000	0.000000	0.071429	0.071429	0.071429	0.142857	0.142857	0.142857	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.071429	0.000000	0.071429	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.071429	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.071429	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.071429	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
71	St. James Town	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020000	0.000000	0.00	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020000	0.0000	0.010000	0.030000	0.010000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.020000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.00	0.030000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.050000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.040000	0.030000	0.070000	0.000000	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.010000	0.00	0.000000	0.030000	0.000000	0.020000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.020000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.010000	0.0	0.000000	0.000000	0.000000	0.020000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.010000	0.010000	0.000000	0.000000	0.000000	0.000000	0.00000	0.010000	0.000000	0.000000	0.000000	0.030000	0.000000	0.000000	0.00	0.01	0.000000	0.00	0.00	0.000000	0.000000	0.010000	0.010000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.010000	0.050000	0.000000	0.000000	0.010000	0.010000	0.000	0.000000	0.000000	0.000000	0.000000	0.030000	0.020000	0.010000	0.000000	0.000000	0.000000	0.010000	0.00	0.010000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.010000	0.00	0.0	0.010000	0.000000	0.000000	0.000000	0.0000	0.010000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.030000	0.010000	0.000000	0.000000	0.000000	0.0	0.020000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.0	0.010000	0.000000	0.000000	0.000000	0.000000	0.060000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.020000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.010000	0.000000	0.00	0.000000	0.000000	0.00	0.010000	0.010000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.010000
72	St. James Town, Cabbagetown	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020833	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.041667	0.020833	0.020833	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.020833	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.020833	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.020833	0.0	0.041667	0.000000	0.000000	0.020833	0.000000	0.020833	0.000000	0.000000	0.000000	0.000000	0.000000	0.083333	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020833	0.000000	0.000000	0.000000	0.020833	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.020833	0.000000	0.020833	0.00	0.00	0.020833	0.00	0.00	0.000000	0.000000	0.020833	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020833	0.000	0.000000	0.000000	0.000000	0.000000	0.041667	0.020833	0.000000	0.020833	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.020833	0.000000	0.000000	0.000000	0.000000	0.041667	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.020833	0.041667	0.000000	0.000000	0.020833	0.020833	0.0	0.041667	0.000000	0.020833	0.020833	0.000000	0.000000	0.000000	0.000000	0.0	0.041667	0.000000	0.000000	0.000000	0.000000	0.062500	0.000000	0.000000	0.000000	0.00000	0.000000	0.020833	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020833	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.020833	0.00	0.000000	0.000000	0.00	0.020833	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
73	Stn A PO Boxes 25 The Esplanade	0.010638	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010638	0.010638	0.00	0.000000	0.021277	0.000000	0.000000	0.000000	0.000000	0.000000	0.010638	0.0000	0.010638	0.021277	0.000000	0.000000	0.000000	0.00	0.00	0.010638	0.000000	0.031915	0.000000	0.010638	0.000000	0.010638	0.000000	0.010638	0.000000	0.00	0.010638	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.042553	0.000000	0.000000	0.000000	0.021277	0.000000	0.000000	0.010638	0.000000	0.010638	0.031915	0.106383	0.000000	0.000000	0.000000	0.00	0.00	0.010638	0.000000	0.010638	0.00	0.000000	0.021277	0.000000	0.021277	0.000000	0.000000	0.000000	0.000000	0.010638	0.000000	0.000000	0.000000	0.010638	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.021277	0.021277	0.00	0.000000	0.000000	0.010638	0.000000	0.000000	0.000	0.000000	0.000000	0.010638	0.010638	0.000000	0.010638	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.010638	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.010638	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.010638	0.031915	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.010638	0.031915	0.021277	0.010638	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.010638	0.010638	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010638	0.00	0.0	0.010638	0.010638	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.010638	0.000000	0.00	0.000000	0.010638	0.000000	0.000000	0.000000	0.000000	0.0	0.010638	0.000000	0.000000	0.000000	0.010638	0.000000	0.000000	0.000000	0.0	0.031915	0.000000	0.000000	0.000000	0.000000	0.053191	0.000000	0.000000	0.000000	0.00000	0.000000	0.010638	0.00	0.000000	0.031915	0.000000	0.010638	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010638	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010638	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.010638	0.000000	0.00	0.000000	0.010638	0.00	0.010638	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
74	Studio District	0.025641	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.051282	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.051282	0.025641	0.025641	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.025641	0.000000	0.00	0.000000	0.025641	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.102564	0.000000	0.000000	0.000000	0.025641	0.025641	0.000000	0.000000	0.000000	0.025641	0.000000	0.076923	0.000000	0.000000	0.000000	0.00	0.00	0.025641	0.000000	0.000000	0.00	0.025641	0.000000	0.025641	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.025641	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.025641	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.025641	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.025641	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.025641	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.051282	0.000000	0.000000	0.000000	0.000000	0.025641	0.000000	0.00	0.025641	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.025641	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.025641	0.000000	0.0000	0.025641	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.025641	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.025641	0.00	0.000000	0.025641	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.025641	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
75	Sullivan, Tam O'Shanter – Sullivan, Clarks Cor...	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.111111	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.111111	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.111111	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.111111	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.111111	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.111111	0.0	0.222222	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.111111	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
76	Summerhill East, Moore Park	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.25	0.000000	0.000000	0.000000	0.000000	0.000000	0.25000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
77	Summerhill West, Forest Hill SE, Deer Park, So...	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.066667	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.066667	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.133333	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.066667	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.066667	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.066667	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.066667	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.066667	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.133333	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.066667	0.000000	0.000000	0.000000	0.000000	0.066667	0.000000	0.066667	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.066667	0.0000	0.000000	0.000000	0.000000
78	Swansea, Runnymede	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.025000	0.000000	0.025000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.050000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.025000	0.000000	0.000000	0.00	0.00	0.025000	0.0	0.075000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.100000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.025000	0.000000	0.025000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.025000	0.0	0.000000	0.000000	0.025000	0.000000	0.000000	0.00	0.000000	0.025000	0.025000	0.000000	0.000000	0.025	0.000000	0.000000	0.000000	0.000000	0.000000	0.025000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.025000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.025000	0.000000	0.000000	0.025000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.025	0.000000	0.000000	0.000000	0.000000	0.050000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.025000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.025000	0.0	0.075000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.025000	0.000000	0.000000	0.000000	0.000000	0.025000	0.000000	0.000000	0.000000	0.00000	0.000000	0.025000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.025000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.050000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.025000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.025000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
79	The Beaches	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.2500	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.250000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.250000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
80	The Beaches West, India Bazaar	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.052632	0.000000	0.000000	0.00	0.052632	0.052632	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.052632	0.00	0.000000	0.052632	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.052632	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.052632	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.052632	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.052632	0.00	0.052632	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.052632	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.105263	0.000000	0.000000	0.052632	0.000000	0.0	0.052632	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.052632	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.052632	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.052632	0.000000	0.000000	0.000000	0.052632	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
81	The Junction North, Runnymede	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.250000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.250000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.250000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
82	The Kingsway, Montgomery Road, Old Mill North	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.333333	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.333333	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.333333	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
83	Thorncliffe Park	0.062500	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.062500	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.062500	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.062500	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.062500	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.062500	0.000000	0.062500	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.125000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.062500	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.062500	0.000000	0.000000	0.000000	0.062500	0.0	0.062500	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.062500	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.062500	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0625	0.000000	0.000000	0.000000
84	Underground city, First Canadian Place	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.040000	0.000000	0.00	0.000000	0.010000	0.000000	0.000000	0.030000	0.000000	0.000000	0.000000	0.0000	0.000000	0.030000	0.000000	0.030000	0.000000	0.00	0.00	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.01	0.000000	0.000000	0.000000	0.000000	0.01	0.020000	0.010000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.080000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.080000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.020000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.030000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.010000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.030000	0.000000	0.000000	0.01	0.00	0.000000	0.01	0.00	0.000000	0.010000	0.000000	0.030000	0.010000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.060000	0.010000	0.000000	0.010000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.01	0.0	0.000000	0.010000	0.000000	0.000000	0.0000	0.010000	0.000000	0.010000	0.000000	0.01	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.020000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.0	0.010000	0.000000	0.000000	0.000000	0.000000	0.050000	0.000000	0.000000	0.000000	0.01000	0.000000	0.010000	0.00	0.000000	0.030000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.030000	0.000000	0.000000	0.000000	0.010000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.010000	0.00	0.010000	0.010000	0.000000	0.000000	0.000000	0.00000	0.010000	0.010000	0.000000	0.000000	0.000000	0.0000	0.010000	0.000000	0.000000
85	Union Station , Harbourfront East, Toronto Isl...	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.05	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.030000	0.010000	0.020000	0.000000	0.02	0.00	0.010000	0.000000	0.010000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.00	0.000000	0.030000	0.000000	0.010000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.040000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.140000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.010000	0.000000	0.000000	0.010000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.020000	0.000000	0.00	0.00	0.000000	0.000000	0.050000	0.010000	0.000000	0.010000	0.010000	0.010	0.000000	0.000000	0.000000	0.000000	0.030000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.01	0.000000	0.000000	0.00	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.01	0.0	0.000000	0.000000	0.000000	0.020000	0.0000	0.010000	0.000000	0.000000	0.010000	0.00	0.000000	0.000000	0.00	0.000000	0.020000	0.010000	0.000000	0.000000	0.000000	0.0	0.040000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.030000	0.000000	0.000000	0.000000	0.01000	0.000000	0.010000	0.03	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020000	0.020000	0.000000	0.000000	0.010000	0.000000	0.010000	0.000000	0.010000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.010000	0.00	0.000000	0.010000	0.000000	0.000000	0.000000	0.00000	0.010000	0.010000	0.000000	0.000000	0.000000	0.0000	0.010000	0.000000	0.000000
86	University of Toronto, Harbord	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.057143	0.000000	0.057143	0.000000	0.00	0.00	0.000000	0.000000	0.028571	0.028571	0.000000	0.000000	0.000000	0.000000	0.057143	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.114286	0.000000	0.000000	0.000000	0.000000	0.028571	0.000000	0.000000	0.000000	0.000000	0.000000	0.057143	0.028571	0.000000	0.028571	0.00	0.00	0.028571	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.028571	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.028571	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.028571	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.028571	0.057143	0.028571	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.028571	0.028571	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.028571	0.0	0.028571	0.000000	0.000000	0.000000	0.000000	0.057143	0.000000	0.000000	0.000000	0.00000	0.000000	0.028571	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.028571	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.028571	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.028571	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
87	Upwood Park, North Park, Maple Leaf Park	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.250000	0.000000	0.000000	0.000000	0.00	0.25	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.25	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
88	Victoria Village	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.250000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.25	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.250000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
89	Westmount	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.142857	0.000000	0.000000	0.000000	0.000000	0.000000	0.142857	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.142857	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.142857	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.285714	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.142857	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
90	Weston	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	1.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
91	Wexford, Maryvale	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.250000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.250000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
92	Willowdale South	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.028571	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.028571	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.057143	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.057143	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.028571	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.028571	0.0	0.000000	0.000000	0.000000	0.000000	0.028571	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.028571	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.028571	0.000000	0.000000	0.028571	0.000000	0.000	0.028571	0.000000	0.000000	0.000000	0.000000	0.057143	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.028571	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.028571	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.028571	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.028571	0.028571	0.0	0.057143	0.000000	0.000000	0.028571	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.085714	0.000000	0.000000	0.000000	0.085714	0.000000	0.000000	0.000000	0.00000	0.000000	0.057143	0.00	0.000000	0.000000	0.000000	0.028571	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.028571	0.000000	0.000000	0.000000	0.028571	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.028571	0.0000	0.000000	0.000000	0.000000
93	Willowdale West	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.200000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.200000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.200000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.200000	0.0	0.200000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
94	Willowdale, Newtonbrook	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.500000	0.000000	0.000000	0.000000	0.000000	0.5	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
95	Woburn	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.100000	0.000000	0.000000	0.0000	0.000000	0.100000	0.100000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.100000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.200000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.100000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.1	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.100000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.100000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
96	Woodbine Gardens, Parkview Hill	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.071429	0.000000	0.000000	0.0000	0.000000	0.000000	0.071429	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.071429	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.071429	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.142857	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.071429	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.071429	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.071429	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.071429	0.071429	0.0	0.142857	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.071429	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
97	Woodbine Heights	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.111111	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.111111	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.111111	0.000000	0.000000	0.000000	0.000000	0.111111	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.111111	0.000000	0.000000	0.000000	0.111111	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.222222	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.111111	0.000000	0.0000	0.000000	0.000000	0.000000
98	York Mills West	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.500000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.500000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
99	Yorkville, The Annex, North Midtown	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.043478	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.043478	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.043478	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.0	0.130435	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.130435	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.043478	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.0000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.043478	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.043478	0.000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.043478	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.043478	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.0	0.000000	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.043478	0.000000	0.000000	0.000000	0.043478	0.0	0.086957	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.043478	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.130435	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.043478	0.000000	0.000000	0.000000	0.0000	0.000000	0.000000	0.000000
Let's confirm the new size
In [22]:
toronto_grouped.shape
Out[22]:
(100, 279)
Let's print each neighborhood along with the top 5 most common venues
In [23]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
----Adelaide, Richmond, King----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3  American Restaurant  0.04
4      Thai Restaurant  0.04


----Agincourt----
               venue  freq
0     Clothing Store  0.25
1     Breakfast Spot  0.25
2             Lounge  0.25
3       Skating Rink  0.25
4  Mobile Phone Shop  0.00


----Agincourt North, Steeles East, L'Amoreaux East, Milliken, Ontario----
                venue  freq
0          Playground   0.5
1                Park   0.5
2         Yoga Studio   0.0
3  Miscellaneous Shop   0.0
4       Movie Theater   0.0


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.22
1        Pharmacy  0.11
2            Pool  0.11
3  Sandwich Place  0.11
4             Pub  0.11


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1   Chinese Restaurant  0.25
2                 Bank  0.25
3                 Café  0.25
4          Yoga Studio  0.00


----Bedford Park, Lawrence Manor East----
                  venue  freq
0    Italian Restaurant  0.12
1           Coffee Shop  0.08
2  Fast Food Restaurant  0.08
3       Thai Restaurant  0.08
4              Pharmacy  0.04


----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1      Restaurant  0.06
2    Cocktail Bar  0.06
3          Bakery  0.04
4  Farmers Market  0.04


----Bloor, Kingsway Park South West, Royal York South West, The Queensway, Mimico----
                    venue  freq
0             Social Club  0.07
1  Thrift / Vintage Store  0.07
2            Burger Joint  0.07
3           Burrito Place  0.07
4    Fast Food Restaurant  0.07


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.06
2       Auto Workshop  0.06
3       Garden Center  0.06
4              Garden  0.06


----CFB Toronto, Downsview East----
               venue  freq
0            Airport  0.25
1           Bus Stop  0.25
2  Electronics Store  0.25
3               Park  0.25
4        Yoga Studio  0.00


----Caledonia-Fairbanks----
                  venue  freq
0                  Park  0.33
1         Women's Store  0.17
2              Pharmacy  0.17
3                Market  0.17
4  Fast Food Restaurant  0.17


----Canada Post Gateway Processing Centre----
                      venue  freq
0                     Hotel  0.18
1               Coffee Shop  0.18
2       American Restaurant  0.09
3      Gym / Fitness Center  0.09
4  Mediterranean Restaurant  0.09


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1                Café  0.06
2  Italian Restaurant  0.05
3        Burger Joint  0.04
4  Chinese Restaurant  0.04


----Christie----
                venue  freq
0       Grocery Store  0.19
1                Café  0.19
2                Park  0.12
3  Italian Restaurant  0.06
4           Nightclub  0.06


----Church and Wellesley----
                 venue  freq
0  Japanese Restaurant  0.07
1          Coffee Shop  0.06
2     Sushi Restaurant  0.06
3              Gay Bar  0.05
4           Restaurant  0.03


----Cliffside West, Birch Cliff----
                   venue  freq
0        College Stadium  0.25
1  General Entertainment  0.25
2           Skating Rink  0.25
3                   Café  0.25
4            Yoga Studio  0.00


----Cliffside, Scarborough Village West, Cliffcrest----
                 venue  freq
0                Motel   0.5
1  American Restaurant   0.5
2          Yoga Studio   0.0
3           Nail Salon   0.0
4          Music Store   0.0


----Commerce Court, Victoria Hotel----
                 venue  freq
0          Coffee Shop  0.10
1                 Café  0.07
2                Hotel  0.06
3           Restaurant  0.06
4  American Restaurant  0.04


----Davisville----
                venue  freq
0        Dessert Shop  0.09
1      Sandwich Place  0.09
2  Italian Restaurant  0.06
3         Coffee Shop  0.06
4                Café  0.06


----Davisville North----
               venue  freq
0     Sandwich Place  0.12
1     Breakfast Spot  0.12
2  Food & Drink Shop  0.12
3              Hotel  0.12
4               Park  0.12


----Design Exchange, Toronto Dominion Centre----
                 venue  freq
0          Coffee Shop  0.15
1                 Café  0.09
2                Hotel  0.08
3           Restaurant  0.04
4  American Restaurant  0.04


----Don Mills North----
                  venue  freq
0   Japanese Restaurant   0.2
1  Caribbean Restaurant   0.2
2                  Pool   0.2
3  Gym / Fitness Center   0.2
4                  Café   0.2


----Don Mills South, Flemingdon Park----
                venue  freq
0                 Gym  0.09
1         Coffee Shop  0.09
2    Asian Restaurant  0.09
3          Beer Store  0.09
4  Italian Restaurant  0.05


----Dovercourt Village, Dufferin----
            venue  freq
0     Supermarket  0.10
1        Pharmacy  0.10
2  Discount Store  0.10
3          Bakery  0.10
4     Music Venue  0.05


----Downsview----
               venue  freq
0      Grocery Store   0.4
1               Bank   0.2
2              Hotel   0.2
3      Shopping Mall   0.2
4  Mobile Phone Shop   0.0


----Downsview Central----
               venue  freq
0   Business Service  0.25
1  Korean Restaurant  0.25
2         Food Truck  0.25
3     Baseball Field  0.25
4        Yoga Studio  0.00


----Downsview North, Wilson Heights, Bathurst Manor----
                  venue  freq
0           Coffee Shop  0.12
1              Pharmacy  0.06
2                 Diner  0.06
3         Shopping Mall  0.06
4  Fast Food Restaurant  0.06


----Downsview Northwest----
                             venue  freq
0               Athletics & Sports  0.25
1               Chinese Restaurant  0.25
2             Gym / Fitness Center  0.25
3                      Coffee Shop  0.25
4  Molecular Gastronomy Restaurant  0.00


----East Toronto----
                        venue  freq
0                        Park  0.50
1           Convenience Store  0.25
2                Intersection  0.25
3                 Yoga Studio  0.00
4  Modern European Restaurant  0.00


----Emery, Humberlea----
                        venue  freq
0              Baseball Field   0.5
1  Construction & Landscaping   0.5
2                 Yoga Studio   0.0
3                      Museum   0.0
4               Movie Theater   0.0


----Exhibition Place, Parkdale Village, Brockton----
                  venue  freq
0        Breakfast Spot  0.11
1                  Café  0.11
2           Coffee Shop  0.11
3                   Bar  0.06
4  Caribbean Restaurant  0.06


----Fairview, Oriole, Henry Farm----
                  venue  freq
0        Clothing Store  0.12
1  Fast Food Restaurant  0.08
2           Coffee Shop  0.06
3      Asian Restaurant  0.03
4              Tea Room  0.03


----Forest Hill West, Forest Hill North----
              venue  freq
0     Jewelry Store   0.2
1             Trail   0.2
2  Sushi Restaurant   0.2
3              Park   0.2
4          Bus Line   0.2


----Glencairn, Ontario----
                 venue  freq
0          Pizza Place  0.33
1     Sushi Restaurant  0.17
2                 Park  0.17
3  Japanese Restaurant  0.17
4                  Pub  0.17


----Golden Mile, Clairlea, Oakridge----
           venue  freq
0         Bakery  0.18
1       Bus Line  0.18
2   Soccer Field  0.09
3    Bus Station  0.09
4  Metro Station  0.09


----Grange Park , Kensington Market, Chinatown----
                           venue  freq
0                           Café  0.07
1                            Bar  0.06
2          Vietnamese Restaurant  0.05
3  Vegetarian / Vegan Restaurant  0.05
4                         Bakery  0.04


----Harbourfront , Regent Park----
         venue  freq
0  Coffee Shop  0.15
1       Bakery  0.06
2         Café  0.06
3         Park  0.06
4          Pub  0.06


----High Park, The Junction South----
                 venue  freq
0                 Park  0.08
1                 Café  0.08
2   Mexican Restaurant  0.08
3  Fried Chicken Joint  0.04
4      Thai Restaurant  0.04


----Highland Creek , Rouge Hill, Port Union----
                        venue  freq
0              History Museum   0.5
1                         Bar   0.5
2  Modern European Restaurant   0.0
3                      Museum   0.0
4               Movie Theater   0.0


----Hillcrest Village----
                      venue  freq
0  Mediterranean Restaurant  0.25
1                      Pool  0.25
2               Golf Course  0.25
3                   Dog Run  0.25
4               Yoga Studio  0.00


----Humber Summit----
                 venue  freq
0          Pizza Place   0.5
1  Empanada Restaurant   0.5
2    Mobile Phone Shop   0.0
3        Movie Theater   0.0
4                Motel   0.0


----Humewood-Cedarvale----
          venue  freq
0  Hockey Arena  0.25
1         Field  0.25
2          Park  0.25
3         Trail  0.25
4   Yoga Studio  0.00


----Ionview, Kennedy Park, East Birchmount Park----
              venue  freq
0    Discount Store  0.33
1       Coffee Shop  0.17
2  Department Store  0.17
3     Train Station  0.17
4        Hobby Shop  0.17


----L'Amoreaux West, Steeles West----
                  venue  freq
0           Coffee Shop  0.12
1  Fast Food Restaurant  0.12
2    Chinese Restaurant  0.12
3          Noodle House  0.06
4           Pizza Place  0.06


----Lawrence Manor, Lawrence Heights----
                   venue  freq
0         Clothing Store  0.33
1          Women's Store  0.07
2               Boutique  0.07
3      Accessories Store  0.07
4  Vietnamese Restaurant  0.07


----Lawrence Park----
                venue  freq
0         Swim School   0.2
1  Dim Sum Restaurant   0.2
2                Park   0.2
3                Lake   0.2
4            Bus Line   0.2


----Leaside----
                 venue  freq
0          Coffee Shop  0.09
1  Sporting Goods Shop  0.09
2         Burger Joint  0.06
3        Grocery Store  0.06
4    Fish & Chips Shop  0.03


----Little Portugal, Trinity–Bellwoods----
              venue  freq
0               Bar  0.13
1       Men's Store  0.06
2  Asian Restaurant  0.05
3       Coffee Shop  0.05
4      Cocktail Bar  0.03


----Malvern, Rouge----
                  venue  freq
0  Fast Food Restaurant   1.0
1           Yoga Studio   0.0
2     Mobile Phone Shop   0.0
3                Museum   0.0
4         Movie Theater   0.0


----Markland Wood, Old Burnhamthorpe, Bloordale Gardens, Eringate----
            venue  freq
0            Café  0.17
1  Shopping Plaza  0.17
2        Pharmacy  0.17
3     Pizza Place  0.17
4      Beer Store  0.17


----Mimico South, New Toronto, Humber Bay Shores----
          venue  freq
0          Café  0.12
1   Pizza Place  0.06
2  Liquor Store  0.06
3     Pet Store  0.06
4   Flower Shop  0.06


----Morningside, Guildwood, West Hill----
                 venue  freq
0          Pizza Place  0.14
1   Mexican Restaurant  0.14
2  Rental Car Location  0.14
3    Electronics Store  0.14
4       Medical Center  0.14


----Mount Olive-Silverstone-Jamestown, South Steeles, Thistletown, Silverstone, Beaumond Heights, Albion Gardens, Humbergate----
                 venue  freq
0        Grocery Store   0.2
1           Beer Store   0.1
2  Fried Chicken Joint   0.1
3         Liquor Store   0.1
4          Coffee Shop   0.1


----North Toronto West----
                 venue  freq
0       Clothing Store  0.14
1  Sporting Goods Shop  0.10
2          Coffee Shop  0.10
3          Yoga Studio  0.05
4        Grocery Store  0.05


----Northwest----
                 venue  freq
0            Drugstore  0.33
1  Rental Car Location  0.33
2                  Bar  0.33
3          Yoga Studio  0.00
4          Music Store  0.00


----Northwood Park, York University----
                venue  freq
0      Massage Studio   0.2
1       Metro Station   0.2
2         Coffee Shop   0.2
3  Miscellaneous Shop   0.2
4                 Bar   0.2


----Old Mill, Sunnylea, Kingsway Park South East, Fairmont Royal York Hotel, Humber Bay, The Queensway, Kingsmills Park, Mimico----
            venue  freq
0  Baseball Field   1.0
1     Yoga Studio   0.0
2          Museum   0.0
3   Movie Theater   0.0
4           Motel   0.0


----Parkdale, Roncesvalles----
                         venue  freq
0               Breakfast Spot  0.12
1                    Gift Shop  0.12
2                    Bookstore  0.06
3                  Coffee Shop  0.06
4  Eastern European Restaurant  0.06


----Parkwoods----
                  venue  freq
0     Food & Drink Shop  0.33
1  Fast Food Restaurant  0.33
2                  Park  0.33
3           Yoga Studio  0.00
4     Mobile Phone Shop  0.00


----Princess Gardens, West Deane Park, Martin Grove, Islington, Cloverdale----
                        venue  freq
0                        Bank   0.5
1                  Print Shop   0.5
2                 Yoga Studio   0.0
3  Modern European Restaurant   0.0
4                      Museum   0.0


----Queen's Park ----
                 venue  freq
0          Coffee Shop  0.23
1                  Gym  0.05
2     Sushi Restaurant  0.05
3                Diner  0.05
4  Japanese Restaurant  0.05


----Richview Gardens, Kingsview Village, St. Phillips, Martin Grove Gardens----
                        venue  freq
0           Mobile Phone Shop  0.33
1                    Bus Line  0.33
2                        Park  0.33
3  Modern European Restaurant  0.00
4                      Museum  0.00


----Riverdale, The Danforth West----
                venue  freq
0    Greek Restaurant  0.24
1         Coffee Shop  0.10
2      Ice Cream Shop  0.07
3  Italian Restaurant  0.05
4           Bookstore  0.05


----Rosedale----
                venue  freq
0                Park  0.50
1          Playground  0.25
2               Trail  0.25
3         Yoga Studio  0.00
4  Miscellaneous Shop  0.00


----Roselawn----
               venue  freq
0             Garden   1.0
1        Yoga Studio   0.0
2  Mobile Phone Shop   0.0
3      Movie Theater   0.0
4              Motel   0.0


----Ryerson, Garden District----
                       venue  freq
0                Coffee Shop  0.09
1             Clothing Store  0.09
2                       Café  0.04
3             Cosmetics Shop  0.03
4  Middle Eastern Restaurant  0.03


----Scarborough Town Centre, Wexford Heights, Dorset Park----
                    venue  freq
0       Indian Restaurant  0.29
1               Pet Store  0.14
2  Furniture / Home Store  0.14
3   Vietnamese Restaurant  0.14
4      Chinese Restaurant  0.14


----Scarborough Village----
                venue  freq
0          Playground   0.5
1          Smoke Shop   0.5
2         Yoga Studio   0.0
3  Miscellaneous Shop   0.0
4       Movie Theater   0.0


----Silver Hills, York Mills----
                        venue  freq
0                   Cafeteria   1.0
1                 Yoga Studio   0.0
2  Modern European Restaurant   0.0
3                 Music Store   0.0
4                      Museum   0.0


----Silverthorn, Mount Dennis, Keelesdale, Del Ray----
                             venue  freq
0                   Discount Store  0.25
1                   Sandwich Place  0.25
2                       Restaurant  0.25
3                      Coffee Shop  0.25
4  Molecular Gastronomy Restaurant  0.00


----South Niagara, Bathurst Quay, King and Spadina, Railway Lands, CN Tower, Harbourfront West, Island airport----
              venue  freq
0    Airport Lounge  0.14
1  Airport Terminal  0.14
2   Airport Service  0.14
3          Boutique  0.07
4  Sculpture Garden  0.07


----St. James Town----
            venue  freq
0     Coffee Shop  0.07
1      Restaurant  0.06
2            Café  0.05
3           Hotel  0.05
4  Clothing Store  0.04


----St. James Town, Cabbagetown----
         venue  freq
0  Coffee Shop  0.08
1   Restaurant  0.06
2       Market  0.04
3         Park  0.04
4         Café  0.04


----Stn A PO Boxes 25 The Esplanade----
         venue  freq
0  Coffee Shop  0.11
1   Restaurant  0.05
2         Café  0.04
3     Beer Bar  0.03
4        Hotel  0.03


----Studio District----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2   Italian Restaurant  0.05
3               Bakery  0.05
4  American Restaurant  0.05


----Sullivan, Tam O'Shanter – Sullivan, Clarks Corners----
                  venue  freq
0           Pizza Place  0.22
1    Chinese Restaurant  0.11
2              Pharmacy  0.11
3  Fast Food Restaurant  0.11
4    Italian Restaurant  0.11


----Summerhill East, Moore Park----
                venue  freq
0               Trail  0.25
1          Playground  0.25
2        Tennis Court  0.25
3                Park  0.25
4  Miscellaneous Shop  0.00


----Summerhill West, Forest Hill SE, Deer Park, South Hill, Rathnelly----
                 venue  freq
0                  Pub  0.13
1          Coffee Shop  0.13
2    Convenience Store  0.07
3  American Restaurant  0.07
4           Sports Bar  0.07


----Swansea, Runnymede----
                venue  freq
0         Coffee Shop  0.10
1                Café  0.08
2         Pizza Place  0.08
3  Italian Restaurant  0.05
4           Bookstore  0.05


----The Beaches----
               venue  freq
0                Pub  0.25
1         Baby Store  0.25
2        Coffee Shop  0.25
3  Mobile Phone Shop  0.00
4             Museum  0.00


----The Beaches West, India Bazaar----
                venue  freq
0                Park  0.11
1           Pet Store  0.05
2  Italian Restaurant  0.05
3        Burger Joint  0.05
4       Burrito Place  0.05


----The Junction North, Runnymede----
                  venue  freq
0           Pizza Place  0.25
1  Caribbean Restaurant  0.25
2                Bakery  0.25
3              Bus Line  0.25
4           Music Venue  0.00


----The Kingsway, Montgomery Road, Old Mill North----
                venue  freq
0               River  0.33
1                Park  0.33
2          Smoke Shop  0.33
3         Yoga Studio  0.00
4  Miscellaneous Shop  0.00


----Thorncliffe Park----
               venue  freq
0  Indian Restaurant  0.12
1        Yoga Studio  0.06
2      Grocery Store  0.06
3               Park  0.06
4        Coffee Shop  0.06


----Underground city, First Canadian Place----
                 venue  freq
0                 Café  0.08
1          Coffee Shop  0.08
2                Hotel  0.06
3           Restaurant  0.05
4  American Restaurant  0.04


----Union Station , Harbourfront East, Toronto Islands----
         venue  freq
0  Coffee Shop  0.14
1        Hotel  0.05
2     Aquarium  0.05
3  Pizza Place  0.04
4         Café  0.04


----University of Toronto, Harbord----
                 venue  freq
0                 Café  0.11
1               Bakery  0.06
2          Coffee Shop  0.06
3            Bookstore  0.06
4  Japanese Restaurant  0.06


----Upwood Park, North Park, Maple Leaf Park----
                        venue  freq
0  Construction & Landscaping  0.25
1                      Bakery  0.25
2                        Park  0.25
3            Basketball Court  0.25
4                 Yoga Studio  0.00


----Victoria Village----
                   venue  freq
0           Hockey Arena  0.25
1  Portuguese Restaurant  0.25
2            Coffee Shop  0.25
3           Intersection  0.25
4            Yoga Studio  0.00


----Westmount----
                       venue  freq
0                Pizza Place  0.29
1             Sandwich Place  0.14
2               Intersection  0.14
3  Middle Eastern Restaurant  0.14
4                Coffee Shop  0.14


----Weston----
               venue  freq
0  Convenience Store   1.0
1        Yoga Studio   0.0
2  Mobile Phone Shop   0.0
3             Museum   0.0
4      Movie Theater   0.0


----Wexford, Maryvale----
                       venue  freq
0             Breakfast Spot  0.25
1                     Bakery  0.25
2                 Smoke Shop  0.25
3  Middle Eastern Restaurant  0.25
4                Yoga Studio  0.00


----Willowdale South----
                 venue  freq
0     Ramen Restaurant  0.09
1           Restaurant  0.09
2  Japanese Restaurant  0.06
3          Pizza Place  0.06
4          Coffee Shop  0.06


----Willowdale West----
           venue  freq
0    Pizza Place   0.2
1       Pharmacy   0.2
2    Coffee Shop   0.2
3  Grocery Store   0.2
4        Butcher   0.2


----Willowdale, Newtonbrook----
                venue  freq
0           Piano Bar   0.5
1                Park   0.5
2  Miscellaneous Shop   0.0
3       Movie Theater   0.0
4               Motel   0.0


----Woburn----
                 venue  freq
0          Coffee Shop   0.2
1  Fried Chicken Joint   0.1
2                 Bank   0.1
3    Korean Restaurant   0.1
4   Athletics & Sports   0.1


----Woodbine Gardens, Parkview Hill----
                  venue  freq
0           Pizza Place  0.14
1  Fast Food Restaurant  0.14
2             Gastropub  0.07
3              Pharmacy  0.07
4             Pet Store  0.07


----Woodbine Heights----
              venue  freq
0      Skating Rink  0.22
1          Pharmacy  0.11
2    Cosmetics Shop  0.11
3  Asian Restaurant  0.11
4       Video Store  0.11


----York Mills West----
                        venue  freq
0                        Bank   0.5
1                        Park   0.5
2                 Yoga Studio   0.0
3  Modern European Restaurant   0.0
4                      Museum   0.0


----Yorkville, The Annex, North Midtown----
            venue  freq
0     Coffee Shop  0.13
1  Sandwich Place  0.13
2            Café  0.13
3     Pizza Place  0.09
4       BBQ Joint  0.04


Let's put that into a pandas dataframe
First, let's write a function to sort the venues in descending order.
In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
Now let's create the new dataframe and display the top 10 venues for each neighborhood.
In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()
Out[36]:
Neighborhood	1st Most Common Venue	2nd Most Common Venue	3rd Most Common Venue	4th Most Common Venue	5th Most Common Venue	6th Most Common Venue	7th Most Common Venue	8th Most Common Venue	9th Most Common Venue	10th Most Common Venue
0	Adelaide, Richmond, King	Coffee Shop	Café	Thai Restaurant	American Restaurant	Steakhouse	Gym	Bar	Bakery	Clothing Store	Hotel
1	Agincourt	Lounge	Skating Rink	Clothing Store	Breakfast Spot	Women's Store	Empanada Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restaurant
2	Agincourt North, Steeles East, L'Amoreaux East...	Playground	Park	Women's Store	Electronics Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restaurant
3	Alderwood, Long Branch	Pizza Place	Pharmacy	Gym	Skating Rink	Coffee Shop	Sandwich Place	Pub	Pool	Doner Restaurant	Dim Sum Restaurant
4	Bayview Village	Japanese Restaurant	Café	Chinese Restaurant	Bank	Dumpling Restaurant	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Women's Store
4. Cluster Neighborhoods
Run k-means to cluster the neighborhood into 5 clusters.
In [37]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]
Out[37]:
array([0, 0, 2, 0, 0, 0, 0, 0, 0, 2], dtype=int32)
Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
In [38]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')



# check the last columns!
toronto_merged.head()
Out[38]:
Postcode	Borough	Neighborhood	Latitude	Longitude	Cluster Labels	1st Most Common Venue	2nd Most Common Venue	3rd Most Common Venue	4th Most Common Venue	5th Most Common Venue	6th Most Common Venue	7th Most Common Venue	8th Most Common Venue	9th Most Common Venue	10th Most Common Venue
0	M3A	North York	Parkwoods	43.753259	-79.329656	2.0	Fast Food Restaurant	Park	Food & Drink Shop	Women's Store	Eastern European Restaurant	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant
1	M4A	North York	Victoria Village	43.725882	-79.315572	0.0	Intersection	Coffee Shop	Hockey Arena	Portuguese Restaurant	Event Space	Falafel Restaurant	Farmers Market	Ethiopian Restaurant	Empanada Restaurant	Dim Sum Restaurant
2	M5A	Downtown Toronto	Harbourfront , Regent Park	43.654260	-79.360636	0.0	Coffee Shop	Pub	Bakery	Park	Café	Breakfast Spot	Restaurant	Mexican Restaurant	Theater	Italian Restaurant
3	M6A	North York	Lawrence Manor, Lawrence Heights	43.718518	-79.464763	0.0	Clothing Store	Women's Store	Coffee Shop	Event Space	Miscellaneous Shop	Fraternity House	Furniture / Home Store	Arts & Crafts Store	Boutique	Accessories Store
4	M7A	Queen's Park (Toronto)	Queen's Park	43.662301	-79.389494	0.0	Coffee Shop	Gym	Sushi Restaurant	Japanese Restaurant	Diner	Yoga Studio	Chinese Restaurant	Smoothie Shop	Seafood Restaurant	Sandwich Place
Finally, let's visualize the resulting clusters
In [64]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters
Out[64]:
5. Examine Clusters
Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.
Cluster 1
In [65]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
Out[65]:
Borough	Cluster Labels	1st Most Common Venue	2nd Most Common Venue	3rd Most Common Venue	4th Most Common Venue	5th Most Common Venue	6th Most Common Venue	7th Most Common Venue	8th Most Common Venue	9th Most Common Venue	10th Most Common Venue
1	North York	0.0	Intersection	Coffee Shop	Hockey Arena	Portuguese Restaurant	Event Space	Falafel Restaurant	Farmers Market	Ethiopian Restaurant	Empanada Restaurant	Dim Sum Restaurant
2	Downtown Toronto	0.0	Coffee Shop	Pub	Bakery	Park	Café	Breakfast Spot	Restaurant	Mexican Restaurant	Theater	Italian Restaurant
3	North York	0.0	Clothing Store	Women's Store	Coffee Shop	Event Space	Miscellaneous Shop	Fraternity House	Furniture / Home Store	Arts & Crafts Store	Boutique	Accessories Store
4	Queen's Park (Toronto)	0.0	Coffee Shop	Gym	Sushi Restaurant	Japanese Restaurant	Diner	Yoga Studio	Chinese Restaurant	Smoothie Shop	Seafood Restaurant	Sandwich Place
7	North York	0.0	Café	Gym / Fitness Center	Caribbean Restaurant	Pool	Japanese Restaurant	Event Space	Ethiopian Restaurant	Empanada Restaurant	Falafel Restaurant	Diner
8	East York	0.0	Fast Food Restaurant	Pizza Place	Intersection	Gastropub	Rock Climbing Spot	Bank	Café	Athletics & Sports	Breakfast Spot	Pharmacy
9	Downtown Toronto	0.0	Clothing Store	Coffee Shop	Café	Middle Eastern Restaurant	Cosmetics Shop	Restaurant	Bar	Ramen Restaurant	Tea Room	Thai Restaurant
10	North York	0.0	Pizza Place	Japanese Restaurant	Park	Sushi Restaurant	Pub	Women's Store	Drugstore	Discount Store	Dog Run	Doner Restaurant
11	Etobicoke	0.0	Print Shop	Bank	Diner	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restaurant	Women's Store
12	Scarborough, Toronto	0.0	History Museum	Bar	Women's Store	Electronics Store	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restaurant	Empanada Restaurant
13	North York	0.0	Gym	Asian Restaurant	Coffee Shop	Beer Store	Shopping Mall	Restaurant	Japanese Restaurant	Italian Restaurant	Café	Dim Sum Restaurant
14	East York	0.0	Skating Rink	Asian Restaurant	Curling Ice	Pharmacy	Park	Video Store	Beer Store	Cosmetics Shop	Doner Restaurant	Donut Shop
15	Downtown Toronto	0.0	Coffee Shop	Restaurant	Hotel	Café	Clothing Store	Breakfast Spot	Gastropub	Cosmetics Shop	Cocktail Bar	Park
17	Etobicoke	0.0	Café	Pizza Place	Liquor Store	Beer Store	Shopping Plaza	Pharmacy	Event Space	Falafel Restaurant	Ethiopian Restaurant	Diner
18	Scarborough, Toronto	0.0	Electronics Store	Pizza Place	Spa	Rental Car Location	Medical Center	Breakfast Spot	Mexican Restaurant	Dumpling Restaurant	Dog Run	Doner Restaurant
19	East Toronto	0.0	Baby Store	Coffee Shop	Pub	Women's Store	Eastern European Restaurant	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant
20	Downtown Toronto	0.0	Coffee Shop	Cocktail Bar	Restaurant	Farmers Market	Cheese Shop	Steakhouse	Italian Restaurant	Beer Bar	Seafood Restaurant	Pub
22	Scarborough, Toronto	0.0	Coffee Shop	Athletics & Sports	Korean Restaurant	Hakka Restaurant	Bakery	Bank	Caribbean Restaurant	Fried Chicken Joint	Thai Restaurant	Donut Shop
23	East York	0.0	Coffee Shop	Sporting Goods Shop	Grocery Store	Burger Joint	Liquor Store	Electronics Store	Bank	Sports Bar	Furniture / Home Store	Beer Store
24	Downtown Toronto	0.0	Coffee Shop	Café	Italian Restaurant	Burger Joint	Chinese Restaurant	Bar	Spa	Salad Place	Thai Restaurant	Sandwich Place
25	Downtown Toronto	0.0	Café	Grocery Store	Park	Athletics & Sports	Italian Restaurant	Restaurant	Diner	Nightclub	Baby Store	Coffee Shop
26	Scarborough, Toronto	0.0	Coffee Shop	Athletics & Sports	Korean Restaurant	Hakka Restaurant	Bakery	Bank	Caribbean Restaurant	Fried Chicken Joint	Thai Restaurant	Donut Shop
27	North York	0.0	Golf Course	Dog Run	Mediterranean Restaurant	Pool	Electronics Store	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restaurant
28	North York	0.0	Coffee Shop	Pharmacy	Frozen Yogurt Shop	Pizza Place	Deli / Bodega	Restaurant	Diner	Sandwich Place	Bridal Shop	Shopping Mall
29	East York	0.0	Indian Restaurant	Yoga Studio	Supermarket	Grocery Store	Liquor Store	Discount Store	Park	Pizza Place	Coffee Shop	Sandwich Place
30	Downtown Toronto	0.0	Coffee Shop	Café	Thai Restaurant	American Restaurant	Steakhouse	Gym	Bar	Bakery	Clothing Store	Hotel
31	West Toronto	0.0	Pharmacy	Bakery	Discount Store	Supermarket	Gym	Liquor Store	Fast Food Restaurant	Middle Eastern Restaurant	Music Venue	Park
32	Scarborough, Toronto	0.0	Playground	Smoke Shop	Women's Store	Eastern European Restaurant	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant
33	North York	0.0	Clothing Store	Fast Food Restaurant	Coffee Shop	Tea Room	Bus Station	Asian Restaurant	Electronics Store	Restaurant	Food Court	Bakery
34	North York	0.0	Bar	Massage Studio	Coffee Shop	Metro Station	Miscellaneous Shop	Women's Store	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restaurant
36	Downtown Toronto	0.0	Coffee Shop	Hotel	Aquarium	Pizza Place	Café	Italian Restaurant	Brewery	Restaurant	Scenic Lookout	Bakery
37	West Toronto	0.0	Bar	Men's Store	Asian Restaurant	Coffee Shop	Vietnamese Restaurant	Cocktail Bar	Bakery	Restaurant	Café	Pizza Place
38	Scarborough, Toronto	0.0	Discount Store	Department Store	Hobby Shop	Coffee Shop	Train Station	Eastern European Restaurant	Dog Run	Doner Restaurant	Donut Shop	Drugstore
39	North York	0.0	Japanese Restaurant	Café	Chinese Restaurant	Bank	Dumpling Restaurant	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Women's Store
41	East Toronto	0.0	Greek Restaurant	Coffee Shop	Ice Cream Shop	Italian Restaurant	Bookstore	Bubble Tea Shop	Fruit & Vegetable Store	Bakery	Juice Bar	Spa
42	Downtown Toronto	0.0	Coffee Shop	Café	Hotel	Restaurant	American Restaurant	Gastropub	Italian Restaurant	Deli / Bodega	Beer Bar	Burger Joint
43	West Toronto	0.0	Breakfast Spot	Coffee Shop	Café	Burrito Place	Stadium	Bar	Italian Restaurant	Furniture / Home Store	Climbing Gym	Performing Arts Venue
44	Scarborough, Toronto	0.0	Bus Line	Bakery	Intersection	Bus Station	Fast Food Restaurant	Metro Station	Food Truck	Soccer Field	Park	Cuban Restaurant
45	North York	0.0	Cafeteria	Electronics Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restaurant	Empanada Restaurant	Diner
46	North York	0.0	Grocery Store	Bank	Shopping Mall	Hotel	Women's Store	Electronics Store	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant
47	East Toronto	0.0	Park	Pet Store	Burger Joint	Italian Restaurant	Steakhouse	Fish & Chips Shop	Light Rail Station	Fast Food Restaurant	Brewery	Sandwich Place
48	Downtown Toronto	0.0	Coffee Shop	Café	Restaurant	Hotel	American Restaurant	Italian Restaurant	Steakhouse	Bakery	Gym	Seafood Restaurant
50	North York	0.0	Pizza Place	Empanada Restaurant	Eastern European Restaurant	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Electronics Store
51	Scarborough, Toronto	0.0	Motel	American Restaurant	Women's Store	Diner	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restaurant
53	North York	0.0	Baseball Field	Food Truck	Korean Restaurant	Business Service	Falafel Restaurant	Event Space	Ethiopian Restaurant	Farmers Market	Empanada Restaurant	Filipino Restaurant
54	East Toronto	0.0	Café	Coffee Shop	Bakery	Italian Restaurant	American Restaurant	Yoga Studio	Park	Brewery	Seafood Restaurant	Sandwich Place
55	North York	0.0	Italian Restaurant	Coffee Shop	Thai Restaurant	Fast Food Restaurant	Pharmacy	Restaurant	Butcher	Café	Pub	Pizza Place
56	York	0.0	Discount Store	Restaurant	Coffee Shop	Sandwich Place	Dumpling Restaurant	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Women's Store
58	Scarborough, Toronto	0.0	College Stadium	Café	General Entertainment	Skating Rink	Eastern European Restaurant	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant
59	North York	0.0	Restaurant	Ramen Restaurant	Japanese Restaurant	Café	Pizza Place	Sandwich Place	Coffee Shop	Middle Eastern Restaurant	Steakhouse	Indonesian Restaurant
60	North York	0.0	Athletics & Sports	Gym / Fitness Center	Coffee Shop	Chinese Restaurant	Eastern European Restaurant	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore
63	York	0.0	Pizza Place	Caribbean Restaurant	Bus Line	Bakery	Discount Store	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restaurant
64	York	0.0	Convenience Store	Women's Store	Empanada Restaurant	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restaurant	Electronics Store	Ethiopian Restaurant
65	Scarborough, Toronto	0.0	Indian Restaurant	Chinese Restaurant	Pet Store	Vietnamese Restaurant	Latin American Restaurant	Furniture / Home Store	Women's Store	Dog Run	Doner Restaurant	Donut Shop
67	Central Toronto	0.0	Hotel	Food & Drink Shop	Park	Gym	Breakfast Spot	Sandwich Place	Clothing Store	Burger Joint	Doner Restaurant	Donut Shop
69	West Toronto	0.0	Park	Mexican Restaurant	Café	Arts & Crafts Store	Fast Food Restaurant	Bookstore	Sandwich Place	Flea Market	Speakeasy	Cajun / Creole Restaurant
70	Etobicoke	0.0	Pizza Place	Middle Eastern Restaurant	Coffee Shop	Sandwich Place	Chinese Restaurant	Intersection	Ethiopian Restaurant	Empanada Restaurant	Electronics Store	Eastern European Restaurant
71	Scarborough, Toronto	0.0	Smoke Shop	Breakfast Spot	Bakery	Middle Eastern Restaurant	Women's Store	Electronics Store	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restaurant
72	North York	0.0	Pharmacy	Grocery Store	Coffee Shop	Butcher	Pizza Place	Event Space	Ethiopian Restaurant	Empanada Restaurant	Falafel Restaurant	Dim Sum Restaurant
73	Central Toronto	0.0	Clothing Store	Coffee Shop	Sporting Goods Shop	Yoga Studio	Bagel Shop	Grocery Store	Fast Food Restaurant	Mexican Restaurant	Diner	Dessert Shop
74	Central Toronto	0.0	Sandwich Place	Café	Coffee Shop	Pizza Place	Liquor Store	Burger Joint	Jewish Restaurant	BBQ Joint	Pub	American Restaurant
75	West Toronto	0.0	Breakfast Spot	Gift Shop	Restaurant	Dessert Shop	Burger Joint	Eastern European Restaurant	Bar	Bank	Dog Run	Italian Restaurant
76	Mississauga	0.0	Coffee Shop	Hotel	Middle Eastern Restaurant	Burrito Place	Gym / Fitness Center	Fried Chicken Joint	Mediterranean Restaurant	Sandwich Place	American Restaurant	Donut Shop
78	Scarborough, Toronto	0.0	Lounge	Skating Rink	Clothing Store	Breakfast Spot	Women's Store	Empanada Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restaurant
79	Central Toronto	0.0	Dessert Shop	Sandwich Place	Café	Italian Restaurant	Coffee Shop	Sushi Restaurant	Pizza Place	Seafood Restaurant	Diner	Restaurant
80	Downtown Toronto	0.0	Café	Coffee Shop	Bar	Japanese Restaurant	Bookstore	Restaurant	Bakery	Poutine Place	Pub	Chinese Restaurant
81	West Toronto	0.0	Coffee Shop	Pizza Place	Café	Bookstore	Italian Restaurant	Sushi Restaurant	Diner	French Restaurant	Indie Movie Theater	Dessert Shop
82	Scarborough, Toronto	0.0	Pizza Place	Pharmacy	Noodle House	Fried Chicken Joint	Italian Restaurant	Thai Restaurant	Chinese Restaurant	Fast Food Restaurant	Cosmetics Shop	Coworking Space
84	Downtown Toronto	0.0	Café	Bar	Vietnamese Restaurant	Vegetarian / Vegan Restaurant	Dumpling Restaurant	Coffee Shop	Bakery	Chinese Restaurant	Mexican Restaurant	Burger Joint
86	Central Toronto	0.0	Coffee Shop	Pub	Medical Center	Fried Chicken Joint	Supermarket	Bagel Shop	Sports Bar	American Restaurant	Pizza Place	Convenience Store
87	Downtown Toronto	0.0	Airport Lounge	Airport Terminal	Airport Service	Harbor / Marina	Plane	Sculpture Garden	Boutique	Boat or Ferry	Airport Gate	Airport
88	Etobicoke	0.0	Café	Pharmacy	Fried Chicken Joint	Pet Store	Pizza Place	Coffee Shop	Restaurant	Sandwich Place	Seafood Restaurant	Liquor Store
89	Etobicoke	0.0	Grocery Store	Fast Food Restaurant	Coffee Shop	Sandwich Place	Liquor Store	Fried Chicken Joint	Beer Store	Pizza Place	Pharmacy	Ethiopian Restaurant
90	Scarborough, Toronto	0.0	Fast Food Restaurant	Coffee Shop	Chinese Restaurant	Pharmacy	Breakfast Spot	Grocery Store	Japanese Restaurant	Nail Salon	Noodle House	Cosmetics Shop
92	Downtown Toronto	0.0	Coffee Shop	Restaurant	Café	Seafood Restaurant	Pub	Beer Bar	Cocktail Bar	Hotel	Italian Restaurant	Japanese Restaurant
93	Etobicoke	0.0	Pizza Place	Pharmacy	Gym	Skating Rink	Coffee Shop	Sandwich Place	Pub	Pool	Doner Restaurant	Dim Sum Restaurant
94	Etobicoke	0.0	Bar	Rental Car Location	Drugstore	Diner	Dog Run	Doner Restaurant	Donut Shop	Dumpling Restaurant	Eastern European Restaurant	Women's Store
96	Downtown Toronto	0.0	Coffee Shop	Restaurant	Bakery	Pub	Pizza Place	Park	Market	Café	Italian Restaurant	Chinese Restaurant
97	Downtown Toronto	0.0	Café	Coffee Shop	Hotel	Restaurant	American Restaurant	Bakery	Gastropub	Seafood Restaurant	Deli / Bodega	Bar
99	Downtown Toronto	0.0	Japanese Restaurant	Sushi Restaurant	Coffee Shop	Gay Bar	Restaurant	Burger Joint	Café	Gastropub	Fast Food Restaurant	Mediterranean Restaurant
100	East Toronto	0.0	Light Rail Station	Yoga Studio	Garden	Comic Shop	Pizza Place	Recording Studio	Restaurant	Butcher	Burrito Place	Brewery
102	Etobicoke	0.0	Fast Food Restaurant	Social Club	Bakery	Burrito Place	Burger Joint	Sandwich Place	Flower Shop	Supplement Shop	Thrift / Vintage Store	Convenience Store
Cluster 2
In [66]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
Out[66]:
Borough	Cluster Labels	1st Most Common Venue	2nd Most Common Venue	3rd Most Common Venue	4th Most Common Venue	5th Most Common Venue	6th Most Common Venue	7th Most Common Venue	8th Most Common Venue	9th Most Common Venue	10th Most Common Venue
6	Scarborough, Toronto	1.0	Fast Food Restaurant	Women's Store	Electronics Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restaurant	Empanada Restaurant
Cluster 3
In [67]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
Out[67]:
Borough	Cluster Labels	1st Most Common Venue	2nd Most Common Venue	3rd Most Common Venue	4th Most Common Venue	5th Most Common Venue	6th Most Common Venue	7th Most Common Venue	8th Most Common Venue	9th Most Common Venue	10th Most Common Venue
0	North York	2.0	Fast Food Restaurant	Park	Food & Drink Shop	Women's Store	Eastern European Restaurant	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant
16	York	2.0	Field	Park	Hockey Arena	Trail	Farmers Market	Falafel Restaurant	Fast Food Restaurant	Event Space	Ethiopian Restaurant	Diner
21	York	2.0	Park	Women's Store	Market	Fast Food Restaurant	Pharmacy	Grocery Store	Farmers Market	Event Space	Ethiopian Restaurant	Empanada Restaurant
35	East York	2.0	Park	Intersection	Convenience Store	Eastern European Restaurant	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant
40	North York	2.0	Electronics Store	Bus Stop	Park	Airport	Women's Store	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restaurant
49	North York	2.0	Basketball Court	Construction & Landscaping	Park	Bakery	Empanada Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restaurant	Electronics Store
52	North York	2.0	Piano Bar	Park	Eastern European Restaurant	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Electronics Store
61	Central Toronto	2.0	Lake	Dim Sum Restaurant	Park	Swim School	Bus Line	Eastern European Restaurant	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant
66	North York	2.0	Park	Bank	Women's Store	Eastern European Restaurant	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Electronics Store
68	Central Toronto	2.0	Trail	Park	Sushi Restaurant	Bus Line	Jewelry Store	Eastern European Restaurant	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant
77	Etobicoke	2.0	Park	Bus Line	Mobile Phone Shop	Women's Store	Eastern European Restaurant	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Electronics Store
83	Central Toronto	2.0	Park	Playground	Tennis Court	Trail	Women's Store	Dumpling Restaurant	Discount Store	Dog Run	Doner Restaurant	Donut Shop
85	Scarborough, Toronto	2.0	Playground	Park	Women's Store	Electronics Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restaurant
91	Downtown Toronto	2.0	Park	Trail	Playground	Eastern European Restaurant	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant
98	Etobicoke	2.0	River	Park	Smoke Shop	Dumpling Restaurant	Diner	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore
Cluster 4
In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
Out[68]:
Borough	Cluster Labels	1st Most Common Venue	2nd Most Common Venue	3rd Most Common Venue	4th Most Common Venue	5th Most Common Venue	6th Most Common Venue	7th Most Common Venue	8th Most Common Venue	9th Most Common Venue	10th Most Common Venue
57	North York	3.0	Baseball Field	Construction & Landscaping	Empanada Restaurant	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restaurant	Electronics Store	Women's Store
101	Etobicoke	3.0	Baseball Field	Empanada Restaurant	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restaurant	Electronics Store	Women's Store	Discount Store
Cluster 5
In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
Out[69]:
Borough	Cluster Labels	1st Most Common Venue	2nd Most Common Venue	3rd Most Common Venue	4th Most Common Venue	5th Most Common Venue	6th Most Common Venue	7th Most Common Venue	8th Most Common Venue	9th Most Common Venue	10th Most Common Venue
62	Central Toronto	4.0	Garden	Electronics Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restaurant	Women's Store	Discount Store
In [ ]:
